# Setup

In [ ]:
%pip install torch-geometric
%pip install pynvml

In [1]:
import os
import math
import time
import copy
import psutil
import pynvml
import random
import numpy as np
import networkx as nx

import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Subset, SubsetRandomSampler, random_split

import torch_geometric.utils as utils
from torch_geometric.utils import to_networkx
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.datasets import TUDataset, Planetoid
from torch_geometric.loader import DataLoader
from torch_geometric.data import Data

from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import confusion_matrix

if torch.cuda.is_available():
    device = torch.device("cuda")
    torch.cuda.manual_seed(42)
    torch.cuda.manual_seed_all(42)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
else:
    device = torch.device("cpu")
print(f'using {device}')

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

set_seed(42)

using cpu


# Data Preprocessing

In [2]:
dataset_path = '/notebooks/dataset/'


def data_cleansing(dataset):
    # Replace negative values with 0
    dataset[dataset < 0] = 0
    
    # Replace NaN values with 0
    dataset = np.nan_to_num(dataset, nan=0)
    
    return dataset

def check_and_drop_invalid_graphs(graph_dataset):
    num_graphs, num_timepoints, num_nodes, _ = graph_dataset.shape
    num_dimensions = 1
    
    valid_graphs = []

    for i in range(num_graphs):
        is_valid = True
        for t in range(num_timepoints):
            adj_matrix = graph_dataset[i, t, :, :]
            num_edges = np.sum(adj_matrix > 0)
            if num_edges == 0:
                is_valid = False
                break
        
        if is_valid:
            valid_graphs.append(i)
    
    cleaned_dataset = graph_dataset[valid_graphs, :, :, :]
    
    return cleaned_dataset

def convert_to_pyg_data(adj_matrices, inp_features):
    pyg_data_list = []
    
    num_graphs = adj_matrices.shape[0]
    num_timepoints = adj_matrices.shape[1]
    
    for i in range(num_graphs):
        for t in range(num_timepoints):
            adj_matrix = adj_matrices[i, t]
            features = inp_features[i, t]
            
            # Get edge indices
            edge_index = torch.tensor(np.array(adj_matrix.nonzero()), dtype=torch.long)
            
            # Get edge attributes
            edge_attr = torch.tensor(adj_matrix[adj_matrix.nonzero()], dtype=torch.float)
            
            # Node features
            x = torch.tensor(features, dtype=torch.float)
            
            # Label (timepoint)
            y = torch.tensor([t], dtype=torch.long)
            
            pyg_data = Data(x=x, edge_index=edge_index, edge_attr=edge_attr, y=y)
            pyg_data_list.append(pyg_data)
    
    return pyg_data_list

def get_dataset_info(pyg_data_list):
    num_features = pyg_data_list[0].x.shape[1]
    all_labels = torch.cat([data.y for data in pyg_data_list])
    num_classes = torch.unique(all_labels).numel()
    return num_features, num_classes


In [3]:
# General Dataset
mutag_dataset = TUDataset(root=f'/tmp/MUTAG', name='MUTAG', use_node_attr=True)
mutag_num_features = mutag_dataset.num_features
mutag_num_classes = mutag_dataset.num_classes
print(f"MUTAG Dataset, Num Features:{mutag_num_features} Num Classes: {mutag_num_classes}")

proteins_dataset = TUDataset(root=f'/tmp/PROTEINS', name='PROTEINS', use_node_attr=True)
proteins_num_features = proteins_dataset.num_features
proteins_num_classes = proteins_dataset.num_classes
print(f"PROTEINS Dataset, Num Features:{proteins_num_features} Num Classes: {proteins_num_classes}")

dd_dataset = TUDataset(root=f'/tmp/DD', name='DD', use_node_attr=True)
dnd_num_features = dd_dataset.num_features
dnd_num_classes = dd_dataset.num_classes
print(f"DD Dataset, Num Features:{dnd_num_features} Num Classes: {dnd_num_classes}")

enzymes_dataset = TUDataset(root=f'/tmp/ENZYMES', name='ENZYMES', use_node_attr=True)
enzymes_num_features = enzymes_dataset.num_features
enzymes_num_classes = enzymes_dataset.num_classes
print(f"ENZYMES Dataset, Num Features:{enzymes_num_features} Num Classes: {enzymes_num_classes}")


# Connectomic Dataset
## OASIS Dataset
oasis = np.load(dataset_path + 'oasis_adj.npy')
oasis_cleaned = data_cleansing(oasis)
oasis_adj = check_and_drop_invalid_graphs(oasis_cleaned)
oasis_features = np.load(dataset_path + 'oasis_laplacian_features.npy')
oasis_num_samples, oasis_num_classes, oasis_num_nodes, oasis_num_features = oasis_features.shape 
oasis_dataset =  convert_to_pyg_data(oasis_adj, oasis_features)

## EMCI-AD Dataset
emci = np.load(dataset_path + 'emci-ad_adj.npy')
emci_cleaned = data_cleansing(emci)
emci_adj = check_and_drop_invalid_graphs(emci_cleaned)
emci_features = np.load(dataset_path + 'emci-ad_laplacian_features.npy')
emci_num_samples, emci_num_classes, emci_num_nodes, emci_num_features = emci_features.shape 
emci_dataset =  convert_to_pyg_data(emci_adj, emci_features) 

## SLIM160 Dataset
slim160 = np.load(dataset_path + 'slim160_adj.npy')
slim160_cleaned = data_cleansing(slim160)
slim160_adj = check_and_drop_invalid_graphs(slim160_cleaned)
slim160_features = np.load(dataset_path + 'slim160_laplacian_features_8.npy')
slim160_num_samples, slim160_num_classes, slim160_num_nodes, slim160_num_features = slim160_features.shape 
slim160_dataset =  convert_to_pyg_data(slim160_adj, slim160_features) 


MUTAG Dataset, Num Features:7 Num Classes: 2
PROTEINS Dataset, Num Features:4 Num Classes: 2
DD Dataset, Num Features:89 Num Classes: 2
ENZYMES Dataset, Num Features:21 Num Classes: 6


# Models

In [4]:
## GCN Models
class GCN(nn.Module):
    def __init__(self, in_features, out_features, bias=True):
        super(GCN, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = nn.Parameter(torch.FloatTensor(in_features, out_features))
        if bias:
            self.bias = nn.Parameter(torch.FloatTensor(out_features))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters() 

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)

    def forward(self, input, adj):
        support = torch.mm(input, self.weight)
        output = torch.spmm(adj, support)
        if self.bias is not None:
            return output + self.bias
        else:
            return output

    def __repr__(self):
        return self.__class__.__name__ + ' (' \
               + str(self.in_features) + ' -> ' \
               + str(self.out_features) + ')'

class GCN1Layer(torch.nn.Module):
    def __init__(self, num_features, hidden_features, num_classes):
        super(GCN1Layer, self).__init__()
        self.gcn1 = GCN(num_features, hidden_features)
        self.bn1 = nn.BatchNorm1d(hidden_features)

        self.fc = torch.nn.Linear(hidden_features, num_classes)

    def forward(self, x, adj, batch):
        # print(f'x shape: {x.shape}, adj shape: {adj.shape}')
        x = F.relu(self.bn1(self.gcn1(x, adj)))
        x = global_mean_pool(x, batch)  # Global mean pooling
        x = self.fc(x)
        return F.log_softmax(x, dim=1)
    
    def count_parameters(self):
        return sum(p.numel() for p in self.parameters() if p.requires_grad)

class GCN2Layer(torch.nn.Module):
    def __init__(self, num_features, hidden_features, num_classes):
        super(GCN2Layer, self).__init__()
        self.gcn1 = GCN(num_features, hidden_features)
        self.bn1 = nn.BatchNorm1d(hidden_features)
        self.gcn2 = GCN(hidden_features, hidden_features*2)
        self.bn2 = nn.BatchNorm1d(hidden_features*2)

        self.fc = torch.nn.Linear(hidden_features*2, num_classes)

    def forward(self, x, adj, batch):
        x = F.relu(self.bn1(self.gcn1(x, adj)))
        x = F.relu(self.bn2(self.gcn2(x, adj)))
        x = global_mean_pool(x, batch)  # Global mean pooling
        x = self.fc(x)
        return F.log_softmax(x, dim=1)
    
    def count_parameters(self):
        return sum(p.numel() for p in self.parameters() if p.requires_grad)

  
## GCESN
class GCESN_1layer(nn.Module):
    def __init__(self, in_features, hidden_features, num_classes, leaky_rate=0.9, num_iterations=5):
        super(GCESN_1layer, self).__init__()
        self.in_features = in_features
        self.hidden_features = hidden_features
        self.num_classes = num_classes
        
        self.leaky_rate = leaky_rate
        self.num_iterations = num_iterations
        self.spectral_radius = 0.9
        self.fc1 = nn.Linear(hidden_features, hidden_features)
        self.bn = nn.BatchNorm1d(hidden_features)

        self.fc = nn.Linear(self.hidden_features, num_classes)

        # Initialize non-trainable weights with dummy tensors
        self.Win = nn.Parameter(torch.zeros((self.in_features, self.hidden_features)), requires_grad=False)
        self.W = nn.Parameter(torch.zeros((self.hidden_features, self.hidden_features)), requires_grad=False)

    def initialize_weights(self):
        self.Win = nn.Parameter(torch.rand((self.in_features, self.hidden_features)) * 2 - 1, requires_grad=False)
        self.W = nn.Parameter(torch.rand((self.hidden_features, self.hidden_features)) * 2 - 1, requires_grad=False)
        self.adjust_spectral_radius()
        
    def adjust_spectral_radius(self):
        eigenvalues, _ = torch.linalg.eig(self.W)
        rhoW = max(abs(eigenvalues))
        self.W *= self.spectral_radius / rhoW

    def forward(self, x, adj, batch):
        h = torch.mm(x, self.Win)
        for _ in range(self.num_iterations):
            h = (1-self.leaky_rate)*h + self.leaky_rate*(F.relu(torch.mm(adj, torch.mm(h, self.W))))
        h = F.relu(self.bn(self.fc1(h)))

        h = global_mean_pool(h, batch)
        h = self.fc(h)
        return F.log_softmax(h, dim=1)
        
    def count_parameters(self):
        return sum(p.numel() for p in self.parameters() if p.requires_grad)

class GCESN_2layer(nn.Module):
    def __init__(self, in_features, hidden_features, num_classes, leaky_rate=0.9, num_iterations=5):
        super(GCESN_2layer, self).__init__()
        self.in_features = in_features
        self.hidden_features = hidden_features
        self.num_classes = num_classes
        
        self.leaky_rate1 = leaky_rate
        self.leaky_rate2 = leaky_rate - 0.1
        self.num_iterations = num_iterations
        self.spectral_radius = 0.9
        self.fc1 = nn.Linear(hidden_features, 2*hidden_features)
        self.fc2 = nn.Linear(2*hidden_features, 2*hidden_features)
        self.bn = nn.BatchNorm1d(2*hidden_features)
        
        self.fc = nn.Linear(2*self.hidden_features, num_classes)

    def initialize_weights(self):
        self.Win = nn.Parameter(torch.rand((self.in_features, self.hidden_features)) * 2 - 1, requires_grad=False)
        self.W_1 = nn.Parameter(torch.rand((self.hidden_features, self.hidden_features)) * 2 - 1, requires_grad=False)
        self.W_2 = nn.Parameter(torch.rand((2*self.hidden_features, 2*self.hidden_features)) * 2 - 1, requires_grad=False)
        self.adjust_spectral_radius()
        
    def adjust_spectral_radius(self):
        eigenvalues_1, _ = torch.linalg.eig(self.W_1)
        rhoW_1 = max(abs(eigenvalues_1))
        self.W_1 *= self.spectral_radius / rhoW_1

        eigenvalues_2, _ = torch.linalg.eig(self.W_2)
        rhoW_2 = max(abs(eigenvalues_2))
        self.W_2 *= self.spectral_radius / rhoW_2

    def forward(self, x, adj, batch):
        h = torch.mm(x, self.Win)
        for _ in range(self.num_iterations):
            h = (1-self.leaky_rate1)*h + self.leaky_rate1*(F.relu(torch.mm(adj, torch.mm(h, self.W_1))))
        h = self.fc1(h)
        
        for _ in range(self.num_iterations):
            h = (1-self.leaky_rate2)*h + self.leaky_rate2*(F.relu(torch.mm(adj, torch.mm(h, self.W_2))))
        h = F.relu(self.bn(self.fc2(h)))

        h = global_mean_pool(h, batch)
        h = self.fc(h)
        return F.log_softmax(h, dim=1)
    
    def count_parameters(self):
        return sum(p.numel() for p in self.parameters() if p.requires_grad)


## TrainableGCESN Model
class SpectralRadiusOptimizerHook:
    def __init__(self, model):
        self.model = model

    def __call__(self):
        self.model.adjust_spectral_radius()

class TrainableGCESN_1layer(nn.Module):
    def __init__(self, in_features, hidden_features, num_classes, leaky_rate=0.9, num_iterations=5):
        super(TrainableGCESN_1layer, self).__init__()
        self.in_features = in_features
        self.hidden_features = hidden_features
        self.num_classes = num_classes
        
        self.leaky_rate = leaky_rate
        self.num_iterations = num_iterations
        self.spectral_radius = 0.9
        self.Win = nn.Parameter(torch.rand((self.in_features, self.hidden_features)) * 2 - 1)
        self.W = nn.Parameter(torch.rand((self.hidden_features, self.hidden_features)) * 2 - 1)
        self.adjust_spectral_radius()
        
        self.fc1 = nn.Linear(hidden_features, hidden_features)
        self.bn = nn.BatchNorm1d(hidden_features)

        self.fc = nn.Linear(self.hidden_features, num_classes)

    def adjust_spectral_radius(self):
        eigenvalues, _ = torch.linalg.eig(self.W)
        rhoW = max(abs(eigenvalues))
        self.W.data *= self.spectral_radius / rhoW

    def forward(self, x, adj, batch):
        h = torch.mm(x, self.Win)
        for _ in range(self.num_iterations):
            h = (1-self.leaky_rate)*h + self.leaky_rate*(F.relu(torch.mm(adj, torch.mm(h, self.W))))
        h = F.relu(self.bn(self.fc1(h)))
        
        h = global_mean_pool(h, batch)
        h = self.fc(h)
        return F.log_softmax(h, dim=1)
    
    def count_parameters(self):
        return sum(p.numel() for p in self.parameters() if p.requires_grad)

class TrainableGCESN_2layer(nn.Module):
    def __init__(self, in_features, hidden_features, num_classes, leaky_rate=0.9, num_iterations=5):
        super(TrainableGCESN_2layer, self).__init__()
        self.in_features = in_features
        self.hidden_features = hidden_features
        self.num_classes = num_classes
        
        self.leaky_rate1 = leaky_rate
        self.leaky_rate2 = leaky_rate - 0.1
        self.num_iterations = num_iterations
        self.spectral_radius = 0.9
        self.Win_1 = nn.Parameter(torch.rand((self.in_features, self.hidden_features)) * 2 - 1)
        self.W_1 = nn.Parameter(torch.rand((self.hidden_features, self.hidden_features)) * 2 - 1)
        self.W_2 = nn.Parameter(torch.rand((2*self.hidden_features, 2*self.hidden_features)) * 2 - 1)
        self.adjust_spectral_radius()

        self.fc1 = nn.Linear(hidden_features, 2*hidden_features)
        self.fc2 = nn.Linear(2*hidden_features, hidden_features)
        self.bn = nn.BatchNorm1d(hidden_features)

        self.fc = nn.Linear(self.hidden_features, num_classes)
    
    def adjust_spectral_radius(self):
        eigenvalues_1, _ = torch.linalg.eig(self.W_1)
        rhoW_1 = max(abs(eigenvalues_1))
        self.W_1.data *= self.spectral_radius / rhoW_1

        eigenvalues_2, _ = torch.linalg.eig(self.W_2)
        rhoW_2 = max(abs(eigenvalues_2))
        self.W_2.data *= self.spectral_radius / rhoW_2

    def forward(self, x, adj, batch):
        h = torch.mm(x, self.Win_1)
        for _ in range(self.num_iterations):
            h = (1-self.leaky_rate1)*h + self.leaky_rate1*(F.relu(torch.mm(adj, torch.mm(h, self.W_1))))
        h = self.fc1(h)
        
        for _ in range(self.num_iterations):
            h = (1-self.leaky_rate2)*h + self.leaky_rate2*(F.relu(torch.mm(adj, torch.mm(h, self.W_2))))
        h = F.relu(self.bn(self.fc2(h)))

        h = global_mean_pool(h, batch)
        h = self.fc(h)
        return F.log_softmax(h, dim=1)
    
    def count_parameters(self):
        return sum(p.numel() for p in self.parameters() if p.requires_grad)

# Trainings & Evaluations

In [28]:
## Single Functions
def single_train(model, loader, val_loader, lr=0.001, num_epochs=100, patience=5, 
                step_size=50, gamma=0.5, save_path='models/gcn_x.pth', 
                binary_classification=True, is_esn=False):

    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = torch.nn.NLLLoss()
    # criterion = torch.nn.CrossEntropyLoss()
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)
    if is_esn == True:
        spectral_hook = SpectralRadiusOptimizerHook(model)

    training_loss = []
    validation_loss = []
    epoch_time = []
    cpu_usage_percent = []
    memory_usage = []
    gpu_usage = []
    gpu_usage_percent = []

    best_train_loss = float('inf')
    best_val_loss = float('inf')
    epochs_no_improve = 0

    model.to(device)

    # Get the process object for the current process
    process = psutil.Process()

    for epoch in range(num_epochs):
        epoch_loss = 0

        set_seed(42)
        model.train()
        epoch_start_time = time.time()

        # Measure CPU and GPU usage before the epoch
        cpu_usage_before = psutil.cpu_percent(interval=None)
        memory_before = process.memory_info().rss
        if torch.cuda.is_available():
            gpu_usage_before = torch.cuda.memory_allocated(device)
            gpu_util_before = torch.cuda.utilization(device)
        else:
            gpu_usage_before = 0
            gpu_util_before = 0

        for data in loader:
            x, edge_index, batch, y = data.x.to(device), data.edge_index.to(device), data.batch.to(device), data.y.to(device)
            adj_matrix = utils.to_dense_adj(edge_index).squeeze(0).to(device)

            optimizer.zero_grad()
            output = model(x, adj_matrix, batch)
            loss = criterion(output, y)
            loss.backward()
            optimizer.step()
            if is_esn==True:
                spectral_hook()

            epoch_loss += loss.item() * data.num_graphs
        
        scheduler.step()  # Step the learning rate scheduler

        epoch_end_time = time.time()
        epoch_time.append(epoch_end_time - epoch_start_time)
        
        # Measure CPU and GPU usage after the epoch
        cpu_usage_after = psutil.cpu_percent(interval=None)
        memory_after = process.memory_info().rss
        if torch.cuda.is_available():
            gpu_usage_after = torch.cuda.memory_allocated(device)
            gpu_util_after = torch.cuda.utilization(device)
        else:
            gpu_usage_after = 0
            gpu_util_after = 0

        # Calculate average CPU and GPU usage during the epoch
        cpu_usage_percent.append((cpu_usage_before + cpu_usage_after) / 2)
        memory_usage.append((memory_before + memory_after) / 2 / (1024**3))  # Convert to GB
        gpu_usage.append((gpu_usage_before + gpu_usage_after) / 2 / (1024**3))  # Convert to GB
        gpu_usage_percent.append((gpu_util_before + gpu_util_after) / 2)

        training_loss.append(epoch_loss)

        model.eval()
        val_loss = 0
        with torch.no_grad():
            for val_data in val_loader:
                x, edge_index, batch, y = val_data.x.to(device), val_data.edge_index.to(device), val_data.batch.to(device), val_data.y.to(device)
                adj_matrix = utils.to_dense_adj(edge_index).squeeze(0).to(device)
                val_output = model(x, adj_matrix, batch)
                val_loss += criterion(val_output, y).item() * val_data.num_graphs

        validation_loss.append(val_loss)

        # Early stopping logic considering both training and validation loss
        if val_loss < best_val_loss or epoch_loss < best_train_loss:
            if val_loss < best_val_loss:
                best_val_loss = val_loss
            if epoch_loss < best_train_loss:
                best_train_loss = epoch_loss
            epochs_no_improve = 0
            total_epoch = epoch + 1
            torch.save(model.state_dict(), save_path)  # Save the best model
        else:
            epochs_no_improve += 1
            if epochs_no_improve >= patience:
                # print(f'Early stopping at epoch {epoch + 1}')
                total_epoch = epoch + 1
                break

    #     print(f'Epoch {epoch + 1}, Train Loss: {epoch_loss}, Val Loss: {val_loss}')
    #     print(f'Time: {epoch_time[-1]:.2f}s, CPU: {cpu_usage_percent[-1]:.2f}%, Memory: {memory_usage[-1]:.2f}GB, GPU: {gpu_usage[-1]:.2f}GB, GPU Util: {gpu_usage_percent[-1]:.2f}%')

    # plt.plot(training_loss, label='Training Loss')
    # plt.plot(validation_loss, label='Validation Loss')
    # plt.xlabel('Epoch')
    # plt.ylabel('Loss')
    # plt.title('Training and Validation Loss')
    # plt.legend()
    # plt.show()

    # model.load_state_dict(torch.load(save_path))
    # print(f'Model saved to {save_path}')

    avg_epoch_time = np.mean(epoch_time)
    avg_cpu_usage_percent = np.mean(cpu_usage_percent)
    avg_memory_usage = np.mean(memory_usage)
    avg_gpu_usage = np.mean(gpu_usage)
    avg_gpu_usage_percent = np.mean(gpu_usage_percent)
    total_training_time = np.sum(epoch_time)
    max_cpu_usage_percent = np.max(cpu_usage_percent)
    max_memory_usage = np.max(memory_usage)
    max_gpu_usage = np.max(gpu_usage)
    max_gpu_usage_percent = np.max(gpu_usage_percent)

    return total_epoch, total_training_time, avg_memory_usage, avg_gpu_usage, max_memory_usage, max_gpu_usage

def binary_evaluation(y_true, y_pred):
    # Convert to numpy arrays for easier manipulation
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    
    # Calculate True Positives, True Negatives, False Positives, False Negatives
    TP = np.sum((y_true == 1) & (y_pred == 1))
    TN = np.sum((y_true == 0) & (y_pred == 0))
    FP = np.sum((y_true == 0) & (y_pred == 1))
    FN = np.sum((y_true == 1) & (y_pred == 0))
    
    # Calculate metrics
    accuracy = (TP + TN) / (TP + TN + FP + FN)
    sensitivity = TP / (TP + FN) if (TP + FN) != 0 else 0
    specificity = TN / (TN + FP) if (TN + FP) != 0 else 0
    
    return accuracy, sensitivity, specificity

def multiclass_evaluation(y_true, y_pred):
    # Convert to numpy arrays for easier manipulation
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)

    # Calculate confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # print(f'Confusion Matrix:\n{cm}')

    # Calculate metrics
    accuracy = np.trace(cm) / np.sum(cm)
    sensitivity = np.zeros(cm.shape[0])
    specificity = np.zeros(cm.shape[0])
    
    for i in range(cm.shape[0]):
        TP = cm[i, i]
        FN = np.sum(cm[i, :]) - TP
        FP = np.sum(cm[:, i]) - TP
        TN = np.sum(cm) - (TP + FN + FP)

        sensitivity[i] = TP / (TP + FN) if (TP + FN) != 0 else 0
        specificity[i] = TN / (TN + FP) if (TN + FP) != 0 else 0

    avg_sensitivity = np.mean(sensitivity)
    avg_specificity = np.mean(specificity)

    return accuracy, avg_sensitivity, avg_specificity

def inference_performance(model, loader):
    model.eval()
    total_inference_time = 0
    cpu_usage_percent = []
    memory_usage = []
    gpu_usage = []
    gpu_usage_percent = []
    labels = []

    # Get the process object for the current process
    process = psutil.Process()

    for data in loader:
        x, edge_index, batch = data.x.to(device), data.edge_index.to(device), data.batch.to(device)
        adj_matrix = utils.to_dense_adj(edge_index).squeeze(0).to(device)
        
        # Measure CPU and GPU usage before inference
        cpu_usage_before = psutil.cpu_percent(interval=None)
        memory_before = process.memory_info().rss
        if torch.cuda.is_available():
            gpu_usage_before = torch.cuda.memory_allocated(device)
            gpu_util_before = torch.cuda.utilization(device)
        else:
            gpu_usage_before = 0
            gpu_util_before = 0

        start_time = time.time()
        with torch.no_grad():
            output = model(x, adj_matrix, batch)
            labels.append(data.y.cpu().numpy())
        end_time = time.time()

        # Measure CPU and GPU usage after inference
        cpu_usage_after = psutil.cpu_percent(interval=None)
        memory_after = process.memory_info().rss
        if torch.cuda.is_available():
            gpu_usage_after = torch.cuda.memory_allocated(device)
            gpu_util_after = torch.cuda.utilization(device)
        else:
            gpu_usage_after = 0
            gpu_util_after = 0

        # Calculate average CPU and GPU usage during inference
        cpu_usage_percent.append((cpu_usage_before + cpu_usage_after) / 2)
        memory_usage.append((memory_before + memory_after) / 2 / (1024**3))  # Convert to GB
        gpu_usage.append((gpu_usage_before + gpu_usage_after) / 2 / (1024**3))  # Convert to GB
        gpu_usage_percent.append((gpu_util_before + gpu_util_after) / 2)

        # Calculate inference time
        inference_time = end_time - start_time
        total_inference_time += inference_time

    avg_cpu_usage_percent = np.mean(cpu_usage_percent)
    avg_memory_usage = np.mean(memory_usage)
    avg_gpu_usage = np.mean(gpu_usage)
    avg_gpu_usage_percent = np.mean(gpu_usage_percent)
    total_inference_time = total_inference_time / len(loader)  # Average inference time per batch

    print(f'\nAverage Inference Time per Batch: {total_inference_time:.4f}s')
    print(f'Average CPU Usage: {avg_cpu_usage_percent:.2f}%')
    print(f'Average Memory Usage: {avg_memory_usage:.2f}GB')
    print(f'Average GPU Usage: {avg_gpu_usage:.2f}GB')
    print(f'Average GPU Utilization: {avg_gpu_usage_percent:.2f}%')

    return 


## Multiple Runs
def multi_train(model, loader, val_loader, lr=0.001, num_epochs=100, patience=5, 
                step_size=50, gamma=0.5, binary_classification=True):
    
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = torch.nn.NLLLoss()
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)

    training_loss = []
    validation_loss = []
    epoch_time = []
    cpu_usage_percent = []
    memory_usage = []
    gpu_usage = []
    gpu_usage_percent = []

    best_train_loss = float('inf')
    best_val_loss = float('inf')
    epochs_no_improve = 0

    model.to(device)

    # Get the process object for the current process
    process = psutil.Process()

    for epoch in range(num_epochs):
        epoch_loss = 0
        model.train()
        epoch_start_time = time.time()

        # Measure CPU and GPU usage before the epoch
        cpu_usage_before = psutil.cpu_percent(interval=None)
        memory_before = process.memory_info().rss
        if torch.cuda.is_available():
            gpu_usage_before = torch.cuda.memory_allocated(device)
            gpu_util_before = torch.cuda.utilization(device)
        else:
            gpu_usage_before = 0
            gpu_util_before = 0

        for data in loader:
            x, edge_index, batch, y = data.x.to(device), data.edge_index.to(device), data.batch.to(device), data.y.to(device)
            adj_matrix = utils.to_dense_adj(edge_index).squeeze(0).to(device)

            optimizer.zero_grad()
            output = model(x, adj_matrix, batch)
            loss = criterion(output, y) ##+ model.ridge_loss() 
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item() * data.num_graphs
        
        scheduler.step()  # Step the learning rate scheduler

        epoch_end_time = time.time()
        epoch_time.append(epoch_end_time - epoch_start_time)
        
        # Measure CPU and GPU usage after the epoch
        cpu_usage_after = psutil.cpu_percent(interval=None)
        memory_after = process.memory_info().rss
        if torch.cuda.is_available():
            gpu_usage_after = torch.cuda.memory_allocated(device)
            gpu_util_after = torch.cuda.utilization(device)
        else:
            gpu_usage_after = 0
            gpu_util_after = 0

        # Calculate average CPU and GPU usage during the epoch
        cpu_usage_percent.append((cpu_usage_before + cpu_usage_after) / 2)
        memory_usage.append((memory_before + memory_after) / 2 / (1024**3))  # Convert to GB
        gpu_usage.append((gpu_usage_before + gpu_usage_after) / 2 / (1024**3))  # Convert to GB
        gpu_usage_percent.append((gpu_util_before + gpu_util_after) / 2)

        training_loss.append(epoch_loss)

        model.eval()
        val_loss = 0
        with torch.no_grad():
            for val_data in val_loader:
                x, edge_index, batch, y = val_data.x.to(device), val_data.edge_index.to(device), val_data.batch.to(device), val_data.y.to(device)
                adj_matrix = utils.to_dense_adj(edge_index).squeeze(0).to(device)
                val_output = model(x, adj_matrix, batch)
                val_loss += criterion(val_output, y).item() * val_data.num_graphs

        validation_loss.append(val_loss)

        # Early stopping logic considering both training and validation loss
        if val_loss < best_val_loss or epoch_loss < best_train_loss:
            if val_loss < best_val_loss:
                best_val_loss = val_loss
            if epoch_loss < best_train_loss:
                best_train_loss = epoch_loss
            epochs_no_improve = 0
            total_epoch = epoch + 1
        else:
            epochs_no_improve += 1
            if epochs_no_improve >= patience:
                total_epoch = epoch + 1
                break

    total_training_time = np.sum(epoch_time)
    avg_epoch_time = np.mean(epoch_time)

    avg_cpu_usage_percent = np.mean(cpu_usage_percent)
    avg_gpu_usage = np.mean(gpu_usage)
    avg_memory_usage = np.mean(memory_usage)

    max_cpu_usage_percent = np.max(cpu_usage_percent)
    max_gpu_usage = np.max(gpu_usage)
    max_memory_usage = np.max(memory_usage)

    return training_loss[-1], total_epoch, total_training_time, avg_epoch_time, avg_cpu_usage_percent, avg_gpu_usage, avg_memory_usage, max_cpu_usage_percent, max_gpu_usage, max_memory_usage 
   
def multi_test(model, loader, binary_classification=True):
    set_seed(42)
    model.eval()
    y_true = []
    y_pred = []

    start_time = time.time()
    with torch.no_grad():
        for data in loader:
            x, edge_index, batch = data.x.to(device), data.edge_index.to(device), data.batch.to(device)
            adj_matrix = utils.to_dense_adj(edge_index).squeeze(0).to(device)
            output = model(x, adj_matrix, batch)
            pred = output.argmax(dim=1)
            y_true.extend(data.y.tolist())
            y_pred.extend(pred.tolist())
    end_time = time.time()  # End the timer
    testing_time = end_time - start_time

    if binary_classification:
        accuracy, sensitivity, specificity = binary_evaluation(y_true, y_pred)
    else:
        accuracy, sensitivity, specificity = multiclass_evaluation(y_true, y_pred)

    return accuracy, sensitivity, specificity, testing_time


## Cross Validation
def single_train_test_cv(model, dataset, num_folds=3, lr=0.001, num_epochs=100, patience=10, step_size=50, gamma=0.5, 
                         save_path='models/gcn_x_cv.pth', binary_classification=True, is_esn=False):

    kfold = KFold(n_splits=num_folds, shuffle=True, random_state=42)

    fold_results = {
        "accuracy": [],
        "sensitivity": [],
        "specificity": [],
        "epochs": [],
        "training_times": [],
        "testing_times": [],
        "avg_memory_usage": [],
        "avg_gpu_usage": [],
        "max_memory_usage": [],
        "max_gpu_usage": []
    }

    for fold, (train_val_idx, test_idx) in enumerate(kfold.split(dataset)):
        print(f"\nFold {fold+1}/{num_folds}")
        
        set_seed(fold)
        # Create samplers
        train_val_sampler = SubsetRandomSampler(train_val_idx)
        test_sampler = SubsetRandomSampler(test_idx)
        
        # Create data loaders with deterministic behavior
        train_val_loader = DataLoader(dataset, sampler=train_val_sampler, batch_size=32, num_workers=0)
        test_loader = DataLoader(dataset, sampler=test_sampler, batch_size=32, num_workers=0)
        
        # Split train_val indices into train and validation indices
        train_size = int(0.9 * len(train_val_idx))
        val_size = len(train_val_idx) - train_size
        train_indices, val_indices = train_val_idx[:train_size], train_val_idx[train_size:]

        # Create subsets
        train_dataset = Subset(dataset, train_indices)
        val_dataset = Subset(dataset, val_indices)
        
        # Create data loaders
        train_loader = DataLoader(train_dataset, batch_size=32, worker_init_fn=seed_worker, num_workers=0)
        val_loader = DataLoader(val_dataset, batch_size=32, worker_init_fn=seed_worker, num_workers=0)
        
        # Initialize model
        model_copy = copy.deepcopy(model)
        
        # Train the model
        # set_seed(42)
        total_epoch, total_training_time, avg_memory_usage, avg_gpu_usage, max_memory_usage, max_gpu_usage = single_train(
            model_copy, train_loader, val_loader, lr=lr, num_epochs=num_epochs, patience=patience, 
            step_size=step_size, gamma=gamma, save_path=save_path, 
            binary_classification=binary_classification, is_esn=is_esn)
        
        # Test the model
        # set_seed(42)
        accuracy, sensitivity, specificity, testing_time = multi_test(model_copy.to(device), test_loader, binary_classification=binary_classification)

        # Collect metrics for the fold
        fold_results["accuracy"].append(accuracy)
        fold_results["sensitivity"].append(sensitivity)
        fold_results["specificity"].append(specificity)
        fold_results["epochs"].append(total_epoch)
        fold_results["training_times"].append(total_training_time)
        fold_results["testing_times"].append(testing_time)
        fold_results["avg_memory_usage"].append(avg_memory_usage)
        fold_results["avg_gpu_usage"].append(avg_gpu_usage)
        fold_results["max_memory_usage"].append(max_memory_usage)
        fold_results["max_gpu_usage"].append(max_gpu_usage)

        print(f"Fold {fold+1} Results:")
        print(f"   Accuracy: {accuracy:.4f}")
        print(f"   Average Sensitivity (Recall): {sensitivity:.4f}")
        print(f"   Average Specificity: {specificity:.4f}")

    # Compute average metrics across all folds
    avg_metrics = {metric: np.mean(fold_results[metric]) for metric in fold_results}
    std_metrics = {metric: np.std(fold_results[metric]) for metric in fold_results}

    print("\nCross-Validation Results:")
    for metric in avg_metrics:
        avg_value = avg_metrics[metric]
        std_value = std_metrics[metric]
        print(f"{metric}: Mean = {avg_value:.4f} ± {std_value:.2f}")

    return avg_metrics

def multi_train_test_cv(model, dataset, num_runs=3, num_folds=3, lr=0.001, num_epochs=100, patience=10, step_size=50, gamma=0.1, 
                        binary_classification=True, best_model_path='models/best_model.pth'):
    
    kfold = KFold(n_splits=num_folds, shuffle=True, random_state=42)

    overall_results = {
        "accuracy": [],
        "sensitivity": [],
        "specificity": [],
        "training_time": [],
        "testing_time": [],
        "epoch_time": [],
        "num_epoch": [],
        "avg_gpu_usage": [],
        "avg_memory_usage": [],
        "max_gpu_usage": [],
        "max_memory_usage": []
    }

    for run in range(num_runs):
        print(f"\nRun {run+1}/{num_runs}")
        fold_results = {
            "accuracy": [],
            "sensitivity": [],
            "specificity": []
        }

        for fold, (train_val_idx, test_idx) in enumerate(kfold.split(dataset)):
            print(f"Fold {fold+1}/{num_folds}")

            # Set seed for each fold to ensure reproducibility
            set_seed(42 + run + fold)
            
            # Create samplers
            train_val_sampler = SubsetRandomSampler(train_val_idx)
            test_sampler = SubsetRandomSampler(test_idx)

            # Create data loaders
            train_val_loader = DataLoader(dataset, sampler=train_val_sampler, batch_size=32)
            test_loader = DataLoader(dataset, sampler=test_sampler, batch_size=32)

            # Split train_val indices into train and validation indices
            train_size = int(0.9 * len(train_val_idx))
            val_size = len(train_val_idx) - train_size
            train_indices, val_indices = train_val_idx[:train_size], train_val_idx[train_size:]

            # Create subsets
            train_dataset = Subset(dataset, train_indices)
            val_dataset = Subset(dataset, val_indices)

            # Create data loaders
            train_loader = DataLoader(train_dataset, batch_size=32)
            val_loader = DataLoader(val_dataset, batch_size=32)

            # Initialize model
            model_copy = copy.deepcopy(model)
            set_seed(42 + run + fold)  # Ensure reproducibility across models
            model_copy.initialize_weights()

            # Train the model
            loss, total_epoch, training_time, epoch_time, avg_cpu, avg_gpu, avg_memory, max_cpu, max_gpu, max_memory = multi_train(
                model_copy, train_loader, val_loader, lr=lr, num_epochs=num_epochs, patience=patience, 
                step_size=step_size, gamma=gamma)

            # Test the model
            accuracy, sensitivity, specificity, testing_time = multi_test(model_copy.to(device), test_loader, binary_classification=binary_classification)

            # Collect metrics for the fold
            fold_results["accuracy"].append(accuracy)
            fold_results["sensitivity"].append(sensitivity)
            fold_results["specificity"].append(specificity)

            # Collect runtime metrics
            overall_results["training_time"].append(training_time)
            overall_results["testing_time"].append(testing_time)
            overall_results["epoch_time"].append(epoch_time)
            overall_results["num_epoch"].append(total_epoch)
            overall_results["avg_gpu_usage"].append(avg_gpu)
            overall_results["avg_memory_usage"].append(avg_memory)
            overall_results["max_gpu_usage"].append(max_gpu)
            overall_results["max_memory_usage"].append(max_memory)

            print(f"Fold {fold+1} Results:")
            print(f"   Accuracy: {accuracy:.4f}")
            print(f"   Average Sensitivity (Recall): {sensitivity:.4f}")
            print(f"   Average Specificity: {specificity:.4f}")

        # Compute average metrics across all folds for the current run
        avg_metrics = {metric: np.mean(fold_results[metric]) for metric in fold_results}
        std_metrics = {metric: np.std(fold_results[metric]) for metric in fold_results}

        print(f"Run {run+1} Cross-Validation Results:")
        for metric in avg_metrics:
            avg_value = avg_metrics[metric]
            std_value = std_metrics[metric]
            print(f"{metric}: Mean = {avg_value:.4f}, Std = {std_value:.4f}")

        # Store results for each run
        overall_results["accuracy"].append(avg_metrics["accuracy"])
        overall_results["sensitivity"].append(avg_metrics["sensitivity"])
        overall_results["specificity"].append(avg_metrics["specificity"])

    # Compute average values across all runs
    final_avg_metrics = {metric: np.mean(overall_results[metric]) for metric in overall_results if "usage" not in metric}
    final_std_metrics = {metric: np.std(overall_results[metric]) for metric in overall_results if "usage" not in metric}
    max_metrics = {metric: np.max(overall_results[metric]) for metric in overall_results if "usage" not in metric}

    print("\nOverall Results:")
    for metric in final_avg_metrics:
        avg_value = final_avg_metrics[metric]
        std_value = final_std_metrics[metric]
        max_value = max_metrics[metric]
        print(f"{metric}: Mean = {avg_value:.4f} ± {std_value:.2f}, Max = {max_value:.4f}")

    return overall_results

# MUTAG

## GCESN

### Iteration 1

In [11]:
set_seed(42)
gcesn_mutag = GCESN_1layer(mutag_num_features, 2*mutag_num_features, mutag_num_classes, leaky_rate=1, num_iterations=1)
print(gcesn_mutag)
print(f"Total number of trainable parameters: {gcesn_mutag.count_parameters()}\n")

avg_metrics = multi_train_test_cv(gcesn_mutag, mutag_dataset,
                lr=0.01, num_epochs=500, step_size=100, gamma=0.5, 
                num_runs=3, binary_classification=True, 
                best_model_path='models/best_gcesn_1l_mutag.pth')


GCESN_1layer(
  (fc1): Linear(in_features=14, out_features=14, bias=True)
  (bn): BatchNorm1d(14, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc): Linear(in_features=14, out_features=2, bias=True)
)
Total number of trainable parameters: 268


Run 1/3
Fold 1/3
Fold 1 Results:
   Accuracy: 0.8413
   Average Sensitivity (Recall): 0.9268
   Average Specificity: 0.6818
Fold 2/3
Fold 2 Results:
   Accuracy: 0.7778
   Average Sensitivity (Recall): 0.8095
   Average Specificity: 0.7143
Fold 3/3
Fold 3 Results:
   Accuracy: 0.8710
   Average Sensitivity (Recall): 0.8810
   Average Specificity: 0.8500
Run 1 Cross-Validation Results:
accuracy: Mean = 0.8300, Std = 0.0389
sensitivity: Mean = 0.8724, Std = 0.0483
specificity: Mean = 0.7487, Std = 0.0728

Run 2/3
Fold 1/3
Fold 1 Results:
   Accuracy: 0.9048
   Average Sensitivity (Recall): 0.9024
   Average Specificity: 0.9091
Fold 2/3
Fold 2 Results:
   Accuracy: 0.7619
   Average Sensitivity (Recall): 0.7857
   Average Speci

In [6]:
set_seed(42)
gcesn_mutag = GCESN_1layer(mutag_num_features, 2*mutag_num_features, mutag_num_classes, leaky_rate=0.9, num_iterations=1)
print(gcesn_mutag)
print(f"Total number of trainable parameters: {gcesn_mutag.count_parameters()}\n")

avg_metrics = multi_train_test_cv(gcesn_mutag, mutag_dataset,
                lr=0.01, num_epochs=500, step_size=100, gamma=0.5, 
                num_runs=3, binary_classification=True, 
                best_model_path='models/best_gcesn_1l_mutag.pth')


GCESN_1layer(
  (fc1): Linear(in_features=14, out_features=14, bias=True)
  (bn): BatchNorm1d(14, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc): Linear(in_features=14, out_features=2, bias=True)
)
Total number of trainable parameters: 268


Run 1/3
Fold 1/3
Fold 1 Results:
   Accuracy: 0.9048
   Average Sensitivity (Recall): 0.9756
   Average Specificity: 0.7727
Fold 2/3
Fold 2 Results:
   Accuracy: 0.7937
   Average Sensitivity (Recall): 0.8095
   Average Specificity: 0.7619
Fold 3/3
Fold 3 Results:
   Accuracy: 0.9032
   Average Sensitivity (Recall): 0.9048
   Average Specificity: 0.9000
Run 1 Cross-Validation Results:
accuracy: Mean = 0.8672, Std = 0.0520
sensitivity: Mean = 0.8966, Std = 0.0680
specificity: Mean = 0.8115, Std = 0.0627

Run 2/3
Fold 1/3
Fold 1 Results:
   Accuracy: 0.9206
   Average Sensitivity (Recall): 0.9268
   Average Specificity: 0.9091
Fold 2/3
Fold 2 Results:
   Accuracy: 0.7143
   Average Sensitivity (Recall): 0.8571
   Average Speci

In [12]:
set_seed(42)
gcesn_mutag = GCESN_1layer(mutag_num_features, 2*mutag_num_features, mutag_num_classes, leaky_rate=0.8, num_iterations=1)
print(gcesn_mutag)
print(f"Total number of trainable parameters: {gcesn_mutag.count_parameters()}\n")

avg_metrics = multi_train_test_cv(gcesn_mutag, mutag_dataset,
                lr=0.01, num_epochs=500, step_size=100, gamma=0.5, 
                num_runs=3, binary_classification=True, 
                best_model_path='models/best_gcesn_1l_mutag.pth')


GCESN_1layer(
  (fc1): Linear(in_features=14, out_features=14, bias=True)
  (bn): BatchNorm1d(14, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc): Linear(in_features=14, out_features=2, bias=True)
)
Total number of trainable parameters: 268


Run 1/3
Fold 1/3
Fold 1 Results:
   Accuracy: 0.8889
   Average Sensitivity (Recall): 0.9268
   Average Specificity: 0.8182
Fold 2/3
Fold 2 Results:
   Accuracy: 0.8095
   Average Sensitivity (Recall): 0.8333
   Average Specificity: 0.7619
Fold 3/3
Fold 3 Results:
   Accuracy: 0.9032
   Average Sensitivity (Recall): 0.9048
   Average Specificity: 0.9000
Run 1 Cross-Validation Results:
accuracy: Mean = 0.8672, Std = 0.0412
sensitivity: Mean = 0.8883, Std = 0.0399
specificity: Mean = 0.8267, Std = 0.0567

Run 2/3
Fold 1/3
Fold 1 Results:
   Accuracy: 0.9206
   Average Sensitivity (Recall): 0.9268
   Average Specificity: 0.9091
Fold 2/3
Fold 2 Results:
   Accuracy: 0.7619
   Average Sensitivity (Recall): 0.8095
   Average Speci

In [13]:
set_seed(42)
gcesn_mutag = GCESN_1layer(mutag_num_features, 2*mutag_num_features, mutag_num_classes, leaky_rate=0.7, num_iterations=1)
print(gcesn_mutag)
print(f"Total number of trainable parameters: {gcesn_mutag.count_parameters()}\n")

avg_metrics = multi_train_test_cv(gcesn_mutag, mutag_dataset,
                lr=0.01, num_epochs=500, step_size=100, gamma=0.5, 
                num_runs=3, binary_classification=True, 
                best_model_path='models/best_gcesn_1l_mutag.pth')

GCESN_1layer(
  (fc1): Linear(in_features=14, out_features=14, bias=True)
  (bn): BatchNorm1d(14, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc): Linear(in_features=14, out_features=2, bias=True)
)
Total number of trainable parameters: 268


Run 1/3
Fold 1/3
Fold 1 Results:
   Accuracy: 0.8730
   Average Sensitivity (Recall): 0.9756
   Average Specificity: 0.6818
Fold 2/3
Fold 2 Results:
   Accuracy: 0.7937
   Average Sensitivity (Recall): 0.8095
   Average Specificity: 0.7619
Fold 3/3
Fold 3 Results:
   Accuracy: 0.9032
   Average Sensitivity (Recall): 0.9048
   Average Specificity: 0.9000
Run 1 Cross-Validation Results:
accuracy: Mean = 0.8566, Std = 0.0462
sensitivity: Mean = 0.8966, Std = 0.0680
specificity: Mean = 0.7812, Std = 0.0901

Run 2/3
Fold 1/3
Fold 1 Results:
   Accuracy: 0.9206
   Average Sensitivity (Recall): 0.9268
   Average Specificity: 0.9091
Fold 2/3
Fold 2 Results:
   Accuracy: 0.7778
   Average Sensitivity (Recall): 0.8095
   Average Speci

### Iteration 2

In [14]:
set_seed(42)
gcesn_mutag = GCESN_1layer(mutag_num_features, 2*mutag_num_features, mutag_num_classes, leaky_rate=1, num_iterations=2)
print(gcesn_mutag)
print(f"Total number of trainable parameters: {gcesn_mutag.count_parameters()}\n")

avg_metrics = multi_train_test_cv(gcesn_mutag, mutag_dataset,
                lr=0.01, num_epochs=500, step_size=100, gamma=0.5, 
                num_runs=3, binary_classification=True, 
                best_model_path='models/best_gcesn_1l_mutag.pth')


GCESN_1layer(
  (fc1): Linear(in_features=14, out_features=14, bias=True)
  (bn): BatchNorm1d(14, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc): Linear(in_features=14, out_features=2, bias=True)
)
Total number of trainable parameters: 268


Run 1/3
Fold 1/3
Fold 1 Results:
   Accuracy: 0.8095
   Average Sensitivity (Recall): 0.9512
   Average Specificity: 0.5455
Fold 2/3
Fold 2 Results:
   Accuracy: 0.8254
   Average Sensitivity (Recall): 0.8095
   Average Specificity: 0.8571
Fold 3/3
Fold 3 Results:
   Accuracy: 0.9032
   Average Sensitivity (Recall): 0.8810
   Average Specificity: 0.9500
Run 1 Cross-Validation Results:
accuracy: Mean = 0.8460, Std = 0.0409
sensitivity: Mean = 0.8806, Std = 0.0578
specificity: Mean = 0.7842, Std = 0.1730

Run 2/3
Fold 1/3
Fold 1 Results:
   Accuracy: 0.9206
   Average Sensitivity (Recall): 0.9268
   Average Specificity: 0.9091
Fold 2/3
Fold 2 Results:
   Accuracy: 0.8095
   Average Sensitivity (Recall): 0.8333
   Average Speci

In [7]:
set_seed(42)
gcesn_mutag = GCESN_1layer(mutag_num_features, 2*mutag_num_features, mutag_num_classes, leaky_rate=0.9, num_iterations=2)
print(gcesn_mutag)
print(f"Total number of trainable parameters: {gcesn_mutag.count_parameters()}\n")

avg_metrics = multi_train_test_cv(gcesn_mutag, mutag_dataset,
                lr=0.01, num_epochs=500, step_size=100, gamma=0.5, 
                num_runs=3, binary_classification=True, 
                best_model_path='models/best_gcesn_1l_mutag.pth')


GCESN_1layer(
  (fc1): Linear(in_features=14, out_features=14, bias=True)
  (bn): BatchNorm1d(14, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc): Linear(in_features=14, out_features=2, bias=True)
)
Total number of trainable parameters: 268


Run 1/3
Fold 1/3
Fold 1 Results:
   Accuracy: 0.8730
   Average Sensitivity (Recall): 0.9512
   Average Specificity: 0.7273
Fold 2/3
Fold 2 Results:
   Accuracy: 0.8254
   Average Sensitivity (Recall): 0.8571
   Average Specificity: 0.7619
Fold 3/3
Fold 3 Results:
   Accuracy: 0.8387
   Average Sensitivity (Recall): 0.8810
   Average Specificity: 0.7500
Run 1 Cross-Validation Results:
accuracy: Mean = 0.8457, Std = 0.0201
sensitivity: Mean = 0.8964, Std = 0.0399
specificity: Mean = 0.7464, Std = 0.0144

Run 2/3
Fold 1/3
Fold 1 Results:
   Accuracy: 0.8095
   Average Sensitivity (Recall): 0.9268
   Average Specificity: 0.5909
Fold 2/3
Fold 2 Results:
   Accuracy: 0.8254
   Average Sensitivity (Recall): 0.8333
   Average Speci

In [15]:
set_seed(42)
gcesn_mutag = GCESN_1layer(mutag_num_features, 2*mutag_num_features, mutag_num_classes, leaky_rate=0.8, num_iterations=2)
print(gcesn_mutag)
print(f"Total number of trainable parameters: {gcesn_mutag.count_parameters()}\n")

avg_metrics = multi_train_test_cv(gcesn_mutag, mutag_dataset,
                lr=0.01, num_epochs=500, step_size=100, gamma=0.5, 
                num_runs=3, binary_classification=True, 
                best_model_path='models/best_gcesn_1l_mutag.pth')


GCESN_1layer(
  (fc1): Linear(in_features=14, out_features=14, bias=True)
  (bn): BatchNorm1d(14, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc): Linear(in_features=14, out_features=2, bias=True)
)
Total number of trainable parameters: 268


Run 1/3
Fold 1/3
Fold 1 Results:
   Accuracy: 0.9365
   Average Sensitivity (Recall): 0.9756
   Average Specificity: 0.8636
Fold 2/3
Fold 2 Results:
   Accuracy: 0.8254
   Average Sensitivity (Recall): 0.8571
   Average Specificity: 0.7619
Fold 3/3
Fold 3 Results:
   Accuracy: 0.8548
   Average Sensitivity (Recall): 0.8810
   Average Specificity: 0.8000
Run 1 Cross-Validation Results:
accuracy: Mean = 0.8722, Std = 0.0470
sensitivity: Mean = 0.9046, Std = 0.0512
specificity: Mean = 0.8085, Std = 0.0420

Run 2/3
Fold 1/3
Fold 1 Results:
   Accuracy: 0.8571
   Average Sensitivity (Recall): 0.9268
   Average Specificity: 0.7273
Fold 2/3
Fold 2 Results:
   Accuracy: 0.8413
   Average Sensitivity (Recall): 0.8571
   Average Speci

In [16]:
set_seed(42)
gcesn_mutag = GCESN_1layer(mutag_num_features, 2*mutag_num_features, mutag_num_classes, leaky_rate=0.7, num_iterations=2)
print(gcesn_mutag)
print(f"Total number of trainable parameters: {gcesn_mutag.count_parameters()}\n")

avg_metrics = multi_train_test_cv(gcesn_mutag, mutag_dataset,
                lr=0.01, num_epochs=500, step_size=100, gamma=0.5, 
                num_runs=3, binary_classification=True, 
                best_model_path='models/best_gcesn_1l_mutag.pth')


GCESN_1layer(
  (fc1): Linear(in_features=14, out_features=14, bias=True)
  (bn): BatchNorm1d(14, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc): Linear(in_features=14, out_features=2, bias=True)
)
Total number of trainable parameters: 268


Run 1/3
Fold 1/3
Fold 1 Results:
   Accuracy: 0.9206
   Average Sensitivity (Recall): 0.9512
   Average Specificity: 0.8636
Fold 2/3
Fold 2 Results:
   Accuracy: 0.8413
   Average Sensitivity (Recall): 0.8571
   Average Specificity: 0.8095
Fold 3/3
Fold 3 Results:
   Accuracy: 0.8710
   Average Sensitivity (Recall): 0.8810
   Average Specificity: 0.8500
Run 1 Cross-Validation Results:
accuracy: Mean = 0.8776, Std = 0.0327
sensitivity: Mean = 0.8964, Std = 0.0399
specificity: Mean = 0.8411, Std = 0.0230

Run 2/3
Fold 1/3
Fold 1 Results:
   Accuracy: 0.8889
   Average Sensitivity (Recall): 0.9268
   Average Specificity: 0.8182
Fold 2/3
Fold 2 Results:
   Accuracy: 0.8095
   Average Sensitivity (Recall): 0.8333
   Average Speci

### Iteration 3

In [17]:
set_seed(42)
gcesn_mutag = GCESN_1layer(mutag_num_features, 2*mutag_num_features, mutag_num_classes, leaky_rate=1, num_iterations=3)
print(gcesn_mutag)
print(f"Total number of trainable parameters: {gcesn_mutag.count_parameters()}\n")

avg_metrics = multi_train_test_cv(gcesn_mutag, mutag_dataset,
                lr=0.01, num_epochs=500, step_size=100, gamma=0.5, 
                num_runs=3, binary_classification=True, 
                best_model_path='models/best_gcesn_1l_mutag.pth')


GCESN_1layer(
  (fc1): Linear(in_features=14, out_features=14, bias=True)
  (bn): BatchNorm1d(14, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc): Linear(in_features=14, out_features=2, bias=True)
)
Total number of trainable parameters: 268


Run 1/3
Fold 1/3
Fold 1 Results:
   Accuracy: 0.7619
   Average Sensitivity (Recall): 1.0000
   Average Specificity: 0.3182
Fold 2/3
Fold 2 Results:
   Accuracy: 0.8254
   Average Sensitivity (Recall): 0.8095
   Average Specificity: 0.8571
Fold 3/3
Fold 3 Results:
   Accuracy: 0.8226
   Average Sensitivity (Recall): 0.8095
   Average Specificity: 0.8500
Run 1 Cross-Validation Results:
accuracy: Mean = 0.8033, Std = 0.0293
sensitivity: Mean = 0.8730, Std = 0.0898
specificity: Mean = 0.6751, Std = 0.2524

Run 2/3
Fold 1/3
Fold 1 Results:
   Accuracy: 0.8889
   Average Sensitivity (Recall): 0.9512
   Average Specificity: 0.7727
Fold 2/3
Fold 2 Results:
   Accuracy: 0.8095
   Average Sensitivity (Recall): 0.7857
   Average Speci

In [8]:
set_seed(42)
gcesn_mutag = GCESN_1layer(mutag_num_features, 2*mutag_num_features, mutag_num_classes, leaky_rate=0.9, num_iterations=3)
print(gcesn_mutag)
print(f"Total number of trainable parameters: {gcesn_mutag.count_parameters()}\n")

avg_metrics = multi_train_test_cv(gcesn_mutag, mutag_dataset,
                lr=0.01, num_epochs=500, step_size=100, gamma=0.5, 
                num_runs=3, binary_classification=True, 
                best_model_path='models/best_gcesn_1l_mutag.pth')


GCESN_1layer(
  (fc1): Linear(in_features=14, out_features=14, bias=True)
  (bn): BatchNorm1d(14, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc): Linear(in_features=14, out_features=2, bias=True)
)
Total number of trainable parameters: 268


Run 1/3
Fold 1/3
Fold 1 Results:
   Accuracy: 0.7302
   Average Sensitivity (Recall): 1.0000
   Average Specificity: 0.2273
Fold 2/3
Fold 2 Results:
   Accuracy: 0.7937
   Average Sensitivity (Recall): 0.7619
   Average Specificity: 0.8571
Fold 3/3
Fold 3 Results:
   Accuracy: 0.8226
   Average Sensitivity (Recall): 0.8571
   Average Specificity: 0.7500
Run 1 Cross-Validation Results:
accuracy: Mean = 0.7821, Std = 0.0386
sensitivity: Mean = 0.8730, Std = 0.0978
specificity: Mean = 0.6115, Std = 0.2752

Run 2/3
Fold 1/3
Fold 1 Results:
   Accuracy: 0.8889
   Average Sensitivity (Recall): 0.9756
   Average Specificity: 0.7273
Fold 2/3
Fold 2 Results:
   Accuracy: 0.7778
   Average Sensitivity (Recall): 0.7857
   Average Speci

In [18]:
set_seed(42)
gcesn_mutag = GCESN_1layer(mutag_num_features, 2*mutag_num_features, mutag_num_classes, leaky_rate=0.8, num_iterations=3)
print(gcesn_mutag)
print(f"Total number of trainable parameters: {gcesn_mutag.count_parameters()}\n")

avg_metrics = multi_train_test_cv(gcesn_mutag, mutag_dataset,
                lr=0.01, num_epochs=500, step_size=100, gamma=0.5, 
                num_runs=3, binary_classification=True, 
                best_model_path='models/best_gcesn_1l_mutag.pth')

GCESN_1layer(
  (fc1): Linear(in_features=14, out_features=14, bias=True)
  (bn): BatchNorm1d(14, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc): Linear(in_features=14, out_features=2, bias=True)
)
Total number of trainable parameters: 268


Run 1/3
Fold 1/3
Fold 1 Results:
   Accuracy: 0.8095
   Average Sensitivity (Recall): 0.9756
   Average Specificity: 0.5000
Fold 2/3
Fold 2 Results:
   Accuracy: 0.8095
   Average Sensitivity (Recall): 0.8333
   Average Specificity: 0.7619
Fold 3/3
Fold 3 Results:
   Accuracy: 0.8548
   Average Sensitivity (Recall): 0.8810
   Average Specificity: 0.8000
Run 1 Cross-Validation Results:
accuracy: Mean = 0.8246, Std = 0.0214
sensitivity: Mean = 0.8966, Std = 0.0591
specificity: Mean = 0.6873, Std = 0.1334

Run 2/3
Fold 1/3
Fold 1 Results:
   Accuracy: 0.9206
   Average Sensitivity (Recall): 0.9268
   Average Specificity: 0.9091
Fold 2/3
Fold 2 Results:
   Accuracy: 0.8254
   Average Sensitivity (Recall): 0.8571
   Average Speci

In [19]:
set_seed(42)
gcesn_mutag = GCESN_1layer(mutag_num_features, 2*mutag_num_features, mutag_num_classes, leaky_rate=0.7, num_iterations=3)
print(gcesn_mutag)
print(f"Total number of trainable parameters: {gcesn_mutag.count_parameters()}\n")

avg_metrics = multi_train_test_cv(gcesn_mutag, mutag_dataset,
                lr=0.01, num_epochs=500, step_size=100, gamma=0.5, 
                num_runs=3, binary_classification=True, 
                best_model_path='models/best_gcesn_1l_mutag.pth')

GCESN_1layer(
  (fc1): Linear(in_features=14, out_features=14, bias=True)
  (bn): BatchNorm1d(14, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc): Linear(in_features=14, out_features=2, bias=True)
)
Total number of trainable parameters: 268


Run 1/3
Fold 1/3
Fold 1 Results:
   Accuracy: 0.9206
   Average Sensitivity (Recall): 0.9756
   Average Specificity: 0.8182
Fold 2/3
Fold 2 Results:
   Accuracy: 0.7460
   Average Sensitivity (Recall): 0.8571
   Average Specificity: 0.5238
Fold 3/3
Fold 3 Results:
   Accuracy: 0.8226
   Average Sensitivity (Recall): 0.8571
   Average Specificity: 0.7500
Run 1 Cross-Validation Results:
accuracy: Mean = 0.8297, Std = 0.0715
sensitivity: Mean = 0.8966, Std = 0.0558
specificity: Mean = 0.6973, Std = 0.1258

Run 2/3
Fold 1/3
Fold 1 Results:
   Accuracy: 0.7143
   Average Sensitivity (Recall): 1.0000
   Average Specificity: 0.1818
Fold 2/3
Fold 2 Results:
   Accuracy: 0.8254
   Average Sensitivity (Recall): 0.8571
   Average Speci

### Iteration 4

In [20]:
set_seed(42)
gcesn_mutag = GCESN_1layer(mutag_num_features, 2*mutag_num_features, mutag_num_classes, leaky_rate=1, num_iterations=4)
print(gcesn_mutag)
print(f"Total number of trainable parameters: {gcesn_mutag.count_parameters()}\n")

avg_metrics = multi_train_test_cv(gcesn_mutag, mutag_dataset,
                lr=0.01, num_epochs=500, step_size=100, gamma=0.5, 
                num_runs=3, binary_classification=True, 
                best_model_path='models/best_gcesn_1l_mutag.pth')


GCESN_1layer(
  (fc1): Linear(in_features=14, out_features=14, bias=True)
  (bn): BatchNorm1d(14, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc): Linear(in_features=14, out_features=2, bias=True)
)
Total number of trainable parameters: 268


Run 1/3
Fold 1/3
Fold 1 Results:
   Accuracy: 0.9206
   Average Sensitivity (Recall): 0.9756
   Average Specificity: 0.8182
Fold 2/3
Fold 2 Results:
   Accuracy: 0.7937
   Average Sensitivity (Recall): 0.8095
   Average Specificity: 0.7619
Fold 3/3
Fold 3 Results:
   Accuracy: 0.7903
   Average Sensitivity (Recall): 0.7619
   Average Specificity: 0.8500
Run 1 Cross-Validation Results:
accuracy: Mean = 0.8349, Std = 0.0607
sensitivity: Mean = 0.8490, Std = 0.0916
specificity: Mean = 0.8100, Std = 0.0364

Run 2/3
Fold 1/3
Fold 1 Results:
   Accuracy: 0.7460
   Average Sensitivity (Recall): 0.8537
   Average Specificity: 0.5455
Fold 2/3
Fold 2 Results:
   Accuracy: 0.7778
   Average Sensitivity (Recall): 0.7619
   Average Speci

In [9]:
set_seed(42)
gcesn_mutag = GCESN_1layer(mutag_num_features, 2*mutag_num_features, mutag_num_classes, leaky_rate=0.9, num_iterations=4)
print(gcesn_mutag)
print(f"Total number of trainable parameters: {gcesn_mutag.count_parameters()}\n")

avg_metrics = multi_train_test_cv(gcesn_mutag, mutag_dataset,
                lr=0.01, num_epochs=500, step_size=100, gamma=0.5, 
                num_runs=3, binary_classification=True, 
                best_model_path='models/best_gcesn_1l_mutag.pth')


GCESN_1layer(
  (fc1): Linear(in_features=14, out_features=14, bias=True)
  (bn): BatchNorm1d(14, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc): Linear(in_features=14, out_features=2, bias=True)
)
Total number of trainable parameters: 268


Run 1/3
Fold 1/3
Fold 1 Results:
   Accuracy: 0.7937
   Average Sensitivity (Recall): 0.9756
   Average Specificity: 0.4545
Fold 2/3
Fold 2 Results:
   Accuracy: 0.7143
   Average Sensitivity (Recall): 0.5952
   Average Specificity: 0.9524
Fold 3/3
Fold 3 Results:
   Accuracy: 0.7903
   Average Sensitivity (Recall): 0.7619
   Average Specificity: 0.8500
Run 1 Cross-Validation Results:
accuracy: Mean = 0.7661, Std = 0.0367
sensitivity: Mean = 0.7776, Std = 0.1557
specificity: Mean = 0.7523, Std = 0.2147

Run 2/3
Fold 1/3
Fold 1 Results:
   Accuracy: 0.8095
   Average Sensitivity (Recall): 0.9756
   Average Specificity: 0.5000
Fold 2/3
Fold 2 Results:
   Accuracy: 0.7937
   Average Sensitivity (Recall): 0.8095
   Average Speci

In [21]:
set_seed(42)
gcesn_mutag = GCESN_1layer(mutag_num_features, 2*mutag_num_features, mutag_num_classes, leaky_rate=0.8, num_iterations=4)
print(gcesn_mutag)
print(f"Total number of trainable parameters: {gcesn_mutag.count_parameters()}\n")

avg_metrics = multi_train_test_cv(gcesn_mutag, mutag_dataset,
                lr=0.01, num_epochs=500, step_size=100, gamma=0.5, 
                num_runs=3, binary_classification=True, 
                best_model_path='models/best_gcesn_1l_mutag.pth')


GCESN_1layer(
  (fc1): Linear(in_features=14, out_features=14, bias=True)
  (bn): BatchNorm1d(14, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc): Linear(in_features=14, out_features=2, bias=True)
)
Total number of trainable parameters: 268


Run 1/3
Fold 1/3
Fold 1 Results:
   Accuracy: 0.8571
   Average Sensitivity (Recall): 0.9512
   Average Specificity: 0.6818
Fold 2/3
Fold 2 Results:
   Accuracy: 0.7460
   Average Sensitivity (Recall): 0.9048
   Average Specificity: 0.4286
Fold 3/3
Fold 3 Results:
   Accuracy: 0.7903
   Average Sensitivity (Recall): 0.7857
   Average Specificity: 0.8000
Run 1 Cross-Validation Results:
accuracy: Mean = 0.7978, Std = 0.0457
sensitivity: Mean = 0.8806, Std = 0.0697
specificity: Mean = 0.6368, Std = 0.1549

Run 2/3
Fold 1/3
Fold 1 Results:
   Accuracy: 0.9365
   Average Sensitivity (Recall): 0.9512
   Average Specificity: 0.9091
Fold 2/3
Fold 2 Results:
   Accuracy: 0.8254
   Average Sensitivity (Recall): 0.8333
   Average Speci

In [22]:
set_seed(42)
gcesn_mutag = GCESN_1layer(mutag_num_features, 2*mutag_num_features, mutag_num_classes, leaky_rate=0.7, num_iterations=4)
print(gcesn_mutag)
print(f"Total number of trainable parameters: {gcesn_mutag.count_parameters()}\n")

avg_metrics = multi_train_test_cv(gcesn_mutag, mutag_dataset,
                lr=0.01, num_epochs=500, step_size=100, gamma=0.5, 
                num_runs=3, binary_classification=True, 
                best_model_path='models/best_gcesn_1l_mutag.pth')


GCESN_1layer(
  (fc1): Linear(in_features=14, out_features=14, bias=True)
  (bn): BatchNorm1d(14, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc): Linear(in_features=14, out_features=2, bias=True)
)
Total number of trainable parameters: 268


Run 1/3
Fold 1/3
Fold 1 Results:
   Accuracy: 0.9048
   Average Sensitivity (Recall): 0.9512
   Average Specificity: 0.8182
Fold 2/3
Fold 2 Results:
   Accuracy: 0.8413
   Average Sensitivity (Recall): 0.8571
   Average Specificity: 0.8095
Fold 3/3
Fold 3 Results:
   Accuracy: 0.8548
   Average Sensitivity (Recall): 0.8095
   Average Specificity: 0.9500
Run 1 Cross-Validation Results:
accuracy: Mean = 0.8670, Std = 0.0273
sensitivity: Mean = 0.8726, Std = 0.0589
specificity: Mean = 0.8592, Std = 0.0643

Run 2/3
Fold 1/3
Fold 1 Results:
   Accuracy: 0.6508
   Average Sensitivity (Recall): 1.0000
   Average Specificity: 0.0000
Fold 2/3
Fold 2 Results:
   Accuracy: 0.8571
   Average Sensitivity (Recall): 0.8571
   Average Speci

### Iteration 5

In [23]:
set_seed(42)
gcesn_mutag = GCESN_1layer(mutag_num_features, 2*mutag_num_features, mutag_num_classes, leaky_rate=1, num_iterations=5)
print(gcesn_mutag)
print(f"Total number of trainable parameters: {gcesn_mutag.count_parameters()}\n")

avg_metrics = multi_train_test_cv(gcesn_mutag, mutag_dataset,
                lr=0.01, num_epochs=500, step_size=100, gamma=0.5, 
                num_runs=3, binary_classification=True, 
                best_model_path='models/best_gcesn_1l_mutag.pth')

GCESN_1layer(
  (fc1): Linear(in_features=14, out_features=14, bias=True)
  (bn): BatchNorm1d(14, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc): Linear(in_features=14, out_features=2, bias=True)
)
Total number of trainable parameters: 268


Run 1/3
Fold 1/3
Fold 1 Results:
   Accuracy: 0.6984
   Average Sensitivity (Recall): 1.0000
   Average Specificity: 0.1364
Fold 2/3
Fold 2 Results:
   Accuracy: 0.7619
   Average Sensitivity (Recall): 0.7619
   Average Specificity: 0.7619
Fold 3/3
Fold 3 Results:
   Accuracy: 0.7742
   Average Sensitivity (Recall): 0.7619
   Average Specificity: 0.8000
Run 1 Cross-Validation Results:
accuracy: Mean = 0.7448, Std = 0.0332
sensitivity: Mean = 0.8413, Std = 0.1122
specificity: Mean = 0.5661, Std = 0.3043

Run 2/3
Fold 1/3
Fold 1 Results:
   Accuracy: 0.8254
   Average Sensitivity (Recall): 0.8537
   Average Specificity: 0.7727
Fold 2/3
Fold 2 Results:
   Accuracy: 0.8095
   Average Sensitivity (Recall): 0.8095
   Average Speci

In [10]:
set_seed(42)
gcesn_mutag = GCESN_1layer(mutag_num_features, 2*mutag_num_features, mutag_num_classes, leaky_rate=0.9, num_iterations=5)
print(gcesn_mutag)
print(f"Total number of trainable parameters: {gcesn_mutag.count_parameters()}\n")

avg_metrics = multi_train_test_cv(gcesn_mutag, mutag_dataset,
                lr=0.01, num_epochs=500, step_size=100, gamma=0.5, 
                num_runs=3, binary_classification=True, 
                best_model_path='models/best_gcesn_1l_mutag.pth')


GCESN_1layer(
  (fc1): Linear(in_features=14, out_features=14, bias=True)
  (bn): BatchNorm1d(14, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc): Linear(in_features=14, out_features=2, bias=True)
)
Total number of trainable parameters: 268


Run 1/3
Fold 1/3
Fold 1 Results:
   Accuracy: 0.7937
   Average Sensitivity (Recall): 1.0000
   Average Specificity: 0.4091
Fold 2/3
Fold 2 Results:
   Accuracy: 0.7778
   Average Sensitivity (Recall): 0.7857
   Average Specificity: 0.7619
Fold 3/3
Fold 3 Results:
   Accuracy: 0.7742
   Average Sensitivity (Recall): 0.6905
   Average Specificity: 0.9500
Run 1 Cross-Validation Results:
accuracy: Mean = 0.7819, Std = 0.0085
sensitivity: Mean = 0.8254, Std = 0.1294
specificity: Mean = 0.7070, Std = 0.2242

Run 2/3
Fold 1/3
Fold 1 Results:
   Accuracy: 0.8571
   Average Sensitivity (Recall): 0.8537
   Average Specificity: 0.8636
Fold 2/3
Fold 2 Results:
   Accuracy: 0.7778
   Average Sensitivity (Recall): 0.8095
   Average Speci

In [24]:
set_seed(42)
gcesn_mutag = GCESN_1layer(mutag_num_features, 2*mutag_num_features, mutag_num_classes, leaky_rate=0.8, num_iterations=5)
print(gcesn_mutag)
print(f"Total number of trainable parameters: {gcesn_mutag.count_parameters()}\n")

avg_metrics = multi_train_test_cv(gcesn_mutag, mutag_dataset,
                lr=0.01, num_epochs=500, step_size=100, gamma=0.5, 
                num_runs=3, binary_classification=True, 
                best_model_path='models/best_gcesn_1l_mutag.pth')

GCESN_1layer(
  (fc1): Linear(in_features=14, out_features=14, bias=True)
  (bn): BatchNorm1d(14, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc): Linear(in_features=14, out_features=2, bias=True)
)
Total number of trainable parameters: 268


Run 1/3
Fold 1/3
Fold 1 Results:
   Accuracy: 0.8413
   Average Sensitivity (Recall): 0.9756
   Average Specificity: 0.5909
Fold 2/3
Fold 2 Results:
   Accuracy: 0.8254
   Average Sensitivity (Recall): 0.8095
   Average Specificity: 0.8571
Fold 3/3
Fold 3 Results:
   Accuracy: 0.8065
   Average Sensitivity (Recall): 0.8095
   Average Specificity: 0.8000
Run 1 Cross-Validation Results:
accuracy: Mean = 0.8244, Std = 0.0142
sensitivity: Mean = 0.8649, Std = 0.0783
specificity: Mean = 0.7494, Std = 0.1144

Run 2/3
Fold 1/3
Fold 1 Results:
   Accuracy: 0.8095
   Average Sensitivity (Recall): 0.9756
   Average Specificity: 0.5000
Fold 2/3
Fold 2 Results:
   Accuracy: 0.7619
   Average Sensitivity (Recall): 0.7619
   Average Speci

In [25]:
set_seed(42)
gcesn_mutag = GCESN_1layer(mutag_num_features, 2*mutag_num_features, mutag_num_classes, leaky_rate=0.7, num_iterations=5)
print(gcesn_mutag)
print(f"Total number of trainable parameters: {gcesn_mutag.count_parameters()}\n")

avg_metrics = multi_train_test_cv(gcesn_mutag, mutag_dataset,
                lr=0.01, num_epochs=500, step_size=100, gamma=0.5, 
                num_runs=3, binary_classification=True, 
                best_model_path='models/best_gcesn_1l_mutag.pth')

GCESN_1layer(
  (fc1): Linear(in_features=14, out_features=14, bias=True)
  (bn): BatchNorm1d(14, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc): Linear(in_features=14, out_features=2, bias=True)
)
Total number of trainable parameters: 268


Run 1/3
Fold 1/3
Fold 1 Results:
   Accuracy: 0.8889
   Average Sensitivity (Recall): 0.9512
   Average Specificity: 0.7727
Fold 2/3
Fold 2 Results:
   Accuracy: 0.7460
   Average Sensitivity (Recall): 0.9048
   Average Specificity: 0.4286
Fold 3/3
Fold 3 Results:
   Accuracy: 0.8387
   Average Sensitivity (Recall): 0.8333
   Average Specificity: 0.8500
Run 1 Cross-Validation Results:
accuracy: Mean = 0.8245, Std = 0.0592
sensitivity: Mean = 0.8964, Std = 0.0485
specificity: Mean = 0.6838, Std = 0.1832

Run 2/3
Fold 1/3
Fold 1 Results:
   Accuracy: 0.9048
   Average Sensitivity (Recall): 0.9512
   Average Specificity: 0.8182
Fold 2/3
Fold 2 Results:
   Accuracy: 0.7619
   Average Sensitivity (Recall): 0.7381
   Average Speci

## Trainable GCESN

### Iteration 1

In [29]:
set_seed(42)
trainable_gcesn_mutag = TrainableGCESN_1layer(mutag_num_features, 2*mutag_num_features, mutag_num_classes, 
                        leaky_rate=1, num_iterations=1)
print(f"Total number of trainable parameters: {trainable_gcesn_mutag.count_parameters()}\n")
                
avg_metrics = single_train_test_cv(trainable_gcesn_mutag, mutag_dataset,
                lr=0.001, num_epochs=500, step_size=500,
                save_path='models/trainable_gcesn_1l_mutag.pth',
                binary_classification=True, is_esn=True)

Total number of trainable parameters: 562


Fold 1/3
Fold 1 Results:
   Accuracy: 0.8571
   Average Sensitivity (Recall): 0.9024
   Average Specificity: 0.7727

Fold 2/3
Fold 2 Results:
   Accuracy: 0.7302
   Average Sensitivity (Recall): 0.8095
   Average Specificity: 0.5714

Fold 3/3
Fold 3 Results:
   Accuracy: 0.8387
   Average Sensitivity (Recall): 0.8810
   Average Specificity: 0.7500

Cross-Validation Results:
accuracy: Mean = 0.8087 ± 0.06
sensitivity: Mean = 0.8643 ± 0.04
specificity: Mean = 0.6981 ± 0.09
epochs: Mean = 498.6667 ± 0.94
training_times: Mean = 5.6139 ± 0.11
testing_times: Mean = 0.0035 ± 0.00
avg_memory_usage: Mean = 0.2821 ± 0.00
avg_gpu_usage: Mean = 0.0000 ± 0.00
max_memory_usage: Mean = 0.2830 ± 0.00
max_gpu_usage: Mean = 0.0000 ± 0.00


In [30]:
set_seed(42)
trainable_gcesn_mutag = TrainableGCESN_1layer(mutag_num_features, 2*mutag_num_features, mutag_num_classes, 
                        leaky_rate=0.9, num_iterations=1)
print(f"Total number of trainable parameters: {trainable_gcesn_mutag.count_parameters()}\n")
                
avg_metrics = single_train_test_cv(trainable_gcesn_mutag, mutag_dataset,
                lr=0.001, num_epochs=500, step_size=500,
                save_path='models/trainable_gcesn_1l_mutag.pth',
                binary_classification=True, is_esn=True)

Total number of trainable parameters: 562


Fold 1/3
Fold 1 Results:
   Accuracy: 0.7937
   Average Sensitivity (Recall): 0.9756
   Average Specificity: 0.4545

Fold 2/3
Fold 2 Results:
   Accuracy: 0.7937
   Average Sensitivity (Recall): 0.8095
   Average Specificity: 0.7619

Fold 3/3
Fold 3 Results:
   Accuracy: 0.8387
   Average Sensitivity (Recall): 0.8810
   Average Specificity: 0.7500

Cross-Validation Results:
accuracy: Mean = 0.8087 ± 0.02
sensitivity: Mean = 0.8887 ± 0.07
specificity: Mean = 0.6555 ± 0.14
epochs: Mean = 500.0000 ± 0.00
training_times: Mean = 5.7635 ± 0.02
testing_times: Mean = 0.0036 ± 0.00
avg_memory_usage: Mean = 0.2558 ± 0.03
avg_gpu_usage: Mean = 0.0000 ± 0.00
max_memory_usage: Mean = 0.2618 ± 0.03
max_gpu_usage: Mean = 0.0000 ± 0.00


In [31]:
set_seed(42)
trainable_gcesn_mutag = TrainableGCESN_1layer(mutag_num_features, 2*mutag_num_features, mutag_num_classes, 
                        leaky_rate=0.8, num_iterations=1)
print(f"Total number of trainable parameters: {trainable_gcesn_mutag.count_parameters()}\n")
                
avg_metrics = single_train_test_cv(trainable_gcesn_mutag, mutag_dataset,
                lr=0.001, num_epochs=500, step_size=500,
                save_path='models/trainable_gcesn_1l_mutag.pth',
                binary_classification=True, is_esn=True)

Total number of trainable parameters: 562


Fold 1/3
Fold 1 Results:
   Accuracy: 0.8730
   Average Sensitivity (Recall): 0.9268
   Average Specificity: 0.7727

Fold 2/3
Fold 2 Results:
   Accuracy: 0.7460
   Average Sensitivity (Recall): 0.8095
   Average Specificity: 0.6190

Fold 3/3
Fold 3 Results:
   Accuracy: 0.9032
   Average Sensitivity (Recall): 0.9048
   Average Specificity: 0.9000

Cross-Validation Results:
accuracy: Mean = 0.8408 ± 0.07
sensitivity: Mean = 0.8804 ± 0.05
specificity: Mean = 0.7639 ± 0.11
epochs: Mean = 499.3333 ± 0.94
training_times: Mean = 5.7382 ± 0.02
testing_times: Mean = 0.0037 ± 0.00
avg_memory_usage: Mean = 0.1616 ± 0.04
avg_gpu_usage: Mean = 0.0000 ± 0.00
max_memory_usage: Mean = 0.1670 ± 0.04
max_gpu_usage: Mean = 0.0000 ± 0.00


In [32]:
set_seed(42)
trainable_gcesn_mutag = TrainableGCESN_1layer(mutag_num_features, 2*mutag_num_features, mutag_num_classes, 
                        leaky_rate=0.7, num_iterations=1)
print(f"Total number of trainable parameters: {trainable_gcesn_mutag.count_parameters()}\n")
                
avg_metrics = single_train_test_cv(trainable_gcesn_mutag, mutag_dataset,
                lr=0.001, num_epochs=500, step_size=500,
                save_path='models/trainable_gcesn_1l_mutag.pth',
                binary_classification=True, is_esn=True)

Total number of trainable parameters: 562


Fold 1/3
Fold 1 Results:
   Accuracy: 0.8571
   Average Sensitivity (Recall): 0.9024
   Average Specificity: 0.7727

Fold 2/3
Fold 2 Results:
   Accuracy: 0.7778
   Average Sensitivity (Recall): 0.8095
   Average Specificity: 0.7143

Fold 3/3
Fold 3 Results:
   Accuracy: 0.8387
   Average Sensitivity (Recall): 0.8810
   Average Specificity: 0.7500

Cross-Validation Results:
accuracy: Mean = 0.8245 ± 0.03
sensitivity: Mean = 0.8643 ± 0.04
specificity: Mean = 0.7457 ± 0.02
epochs: Mean = 500.0000 ± 0.00
training_times: Mean = 5.7596 ± 0.09
testing_times: Mean = 0.0036 ± 0.00
avg_memory_usage: Mean = 0.1257 ± 0.00
avg_gpu_usage: Mean = 0.0000 ± 0.00
max_memory_usage: Mean = 0.1261 ± 0.00
max_gpu_usage: Mean = 0.0000 ± 0.00


### Iteration 2

In [45]:
set_seed(42)
trainable_gcesn_mutag = TrainableGCESN_1layer(mutag_num_features, 2*mutag_num_features, mutag_num_classes, 
                        leaky_rate=1, num_iterations=2)
print(f"Total number of trainable parameters: {trainable_gcesn_mutag.count_parameters()}\n")
                
avg_metrics = single_train_test_cv(trainable_gcesn_mutag, mutag_dataset,
                lr=0.001, num_epochs=500, step_size=500,
                save_path='models/trainable_gcesn_1l_mutag.pth',
                binary_classification=True, is_esn=True)

Total number of trainable parameters: 562


Fold 1/3
Fold 1 Results:
   Accuracy: 0.7937
   Average Sensitivity (Recall): 0.9756
   Average Specificity: 0.4545

Fold 2/3
Fold 2 Results:
   Accuracy: 0.8095
   Average Sensitivity (Recall): 0.8333
   Average Specificity: 0.7619

Fold 3/3
Fold 3 Results:
   Accuracy: 0.8710
   Average Sensitivity (Recall): 0.8810
   Average Specificity: 0.8500

Cross-Validation Results:
accuracy: Mean = 0.8247 ± 0.03
sensitivity: Mean = 0.8966 ± 0.06
specificity: Mean = 0.6888 ± 0.17
epochs: Mean = 417.3333 ± 94.97
training_times: Mean = 5.1016 ± 1.14
testing_times: Mean = 0.0036 ± 0.00
avg_memory_usage: Mean = 0.1119 ± 0.00
avg_gpu_usage: Mean = 0.0000 ± 0.00
max_memory_usage: Mean = 0.1136 ± 0.00
max_gpu_usage: Mean = 0.0000 ± 0.00


In [46]:
set_seed(42)
trainable_gcesn_mutag = TrainableGCESN_1layer(mutag_num_features, 2*mutag_num_features, mutag_num_classes, 
                        leaky_rate=0.9, num_iterations=2)
print(f"Total number of trainable parameters: {trainable_gcesn_mutag.count_parameters()}\n")
                
avg_metrics = single_train_test_cv(trainable_gcesn_mutag, mutag_dataset,
                lr=0.001, num_epochs=500, step_size=500,
                save_path='models/trainable_gcesn_1l_mutag.pth',
                binary_classification=True, is_esn=True)

Total number of trainable parameters: 562


Fold 1/3
Fold 1 Results:
   Accuracy: 0.9206
   Average Sensitivity (Recall): 0.9756
   Average Specificity: 0.8182

Fold 2/3
Fold 2 Results:
   Accuracy: 0.7937
   Average Sensitivity (Recall): 0.7857
   Average Specificity: 0.8095

Fold 3/3
Fold 3 Results:
   Accuracy: 0.8871
   Average Sensitivity (Recall): 0.8810
   Average Specificity: 0.9000

Cross-Validation Results:
accuracy: Mean = 0.8671 ± 0.05
sensitivity: Mean = 0.8808 ± 0.08
specificity: Mean = 0.8426 ± 0.04
epochs: Mean = 475.0000 ± 30.47
training_times: Mean = 6.0314 ± 0.23
testing_times: Mean = 0.0040 ± 0.00
avg_memory_usage: Mean = 0.0985 ± 0.01
avg_gpu_usage: Mean = 0.0000 ± 0.00
max_memory_usage: Mean = 0.1018 ± 0.01
max_gpu_usage: Mean = 0.0000 ± 0.00


In [47]:
set_seed(42)
trainable_gcesn_mutag = TrainableGCESN_1layer(mutag_num_features, 2*mutag_num_features, mutag_num_classes, 
                        leaky_rate=0.8, num_iterations=2)
print(f"Total number of trainable parameters: {trainable_gcesn_mutag.count_parameters()}\n")
                
avg_metrics = single_train_test_cv(trainable_gcesn_mutag, mutag_dataset,
                lr=0.001, num_epochs=500, step_size=500,
                save_path='models/trainable_gcesn_1l_mutag.pth',
                binary_classification=True, is_esn=True)

Total number of trainable parameters: 562


Fold 1/3
Fold 1 Results:
   Accuracy: 0.8730
   Average Sensitivity (Recall): 0.9756
   Average Specificity: 0.6818

Fold 2/3
Fold 2 Results:
   Accuracy: 0.8254
   Average Sensitivity (Recall): 0.8571
   Average Specificity: 0.7619

Fold 3/3
Fold 3 Results:
   Accuracy: 0.8871
   Average Sensitivity (Recall): 0.9286
   Average Specificity: 0.8000

Cross-Validation Results:
accuracy: Mean = 0.8618 ± 0.03
sensitivity: Mean = 0.9204 ± 0.05
specificity: Mean = 0.7479 ± 0.05
epochs: Mean = 435.6667 ± 90.27
training_times: Mean = 5.8998 ± 1.21
testing_times: Mean = 0.0046 ± 0.00
avg_memory_usage: Mean = 0.0921 ± 0.01
avg_gpu_usage: Mean = 0.0000 ± 0.00
max_memory_usage: Mean = 0.0973 ± 0.01
max_gpu_usage: Mean = 0.0000 ± 0.00


In [48]:
set_seed(42)
trainable_gcesn_mutag = TrainableGCESN_1layer(mutag_num_features, 2*mutag_num_features, mutag_num_classes, 
                        leaky_rate=0.7, num_iterations=2)
print(f"Total number of trainable parameters: {trainable_gcesn_mutag.count_parameters()}\n")
                
avg_metrics = single_train_test_cv(trainable_gcesn_mutag, mutag_dataset,
                lr=0.001, num_epochs=500, step_size=500,
                save_path='models/trainable_gcesn_1l_mutag.pth',
                binary_classification=True, is_esn=True)

Total number of trainable parameters: 562


Fold 1/3
Fold 1 Results:
   Accuracy: 0.9365
   Average Sensitivity (Recall): 0.9756
   Average Specificity: 0.8636

Fold 2/3
Fold 2 Results:
   Accuracy: 0.7619
   Average Sensitivity (Recall): 0.7619
   Average Specificity: 0.7619

Fold 3/3
Fold 3 Results:
   Accuracy: 0.8871
   Average Sensitivity (Recall): 0.9048
   Average Specificity: 0.8500

Cross-Validation Results:
accuracy: Mean = 0.8618 ± 0.07
sensitivity: Mean = 0.8808 ± 0.09
specificity: Mean = 0.8252 ± 0.05
epochs: Mean = 458.0000 ± 57.99
training_times: Mean = 6.2059 ± 1.13
testing_times: Mean = 0.0039 ± 0.00
avg_memory_usage: Mean = 0.0857 ± 0.00
avg_gpu_usage: Mean = 0.0000 ± 0.00
max_memory_usage: Mean = 0.0928 ± 0.00
max_gpu_usage: Mean = 0.0000 ± 0.00


### Iteration 3

In [41]:
set_seed(42)
trainable_gcesn_mutag = TrainableGCESN_1layer(mutag_num_features, 2*mutag_num_features, mutag_num_classes, 
                        leaky_rate=1, num_iterations=3)
print(f"Total number of trainable parameters: {trainable_gcesn_mutag.count_parameters()}\n")
                
avg_metrics = single_train_test_cv(trainable_gcesn_mutag, mutag_dataset,
                lr=0.001, num_epochs=500, step_size=500,
                save_path='models/trainable_gcesn_1l_mutag.pth',
                binary_classification=True, is_esn=True)

Total number of trainable parameters: 562


Fold 1/3
Fold 1 Results:
   Accuracy: 0.8095
   Average Sensitivity (Recall): 1.0000
   Average Specificity: 0.4545

Fold 2/3
Fold 2 Results:
   Accuracy: 0.7619
   Average Sensitivity (Recall): 0.7381
   Average Specificity: 0.8095

Fold 3/3
Fold 3 Results:
   Accuracy: 0.8065
   Average Sensitivity (Recall): 0.8810
   Average Specificity: 0.6500

Cross-Validation Results:
accuracy: Mean = 0.7926 ± 0.02
sensitivity: Mean = 0.8730 ± 0.11
specificity: Mean = 0.6380 ± 0.15
epochs: Mean = 449.3333 ± 47.39
training_times: Mean = 5.9216 ± 0.49
testing_times: Mean = 0.0039 ± 0.00
avg_memory_usage: Mean = 0.1466 ± 0.07
avg_gpu_usage: Mean = 0.0000 ± 0.00
max_memory_usage: Mean = 0.1594 ± 0.08
max_gpu_usage: Mean = 0.0000 ± 0.00


In [42]:
set_seed(42)
trainable_gcesn_mutag = TrainableGCESN_1layer(mutag_num_features, 2*mutag_num_features, mutag_num_classes, 
                        leaky_rate=0.9, num_iterations=3)
print(f"Total number of trainable parameters: {trainable_gcesn_mutag.count_parameters()}\n")
                
avg_metrics = single_train_test_cv(trainable_gcesn_mutag, mutag_dataset,
                lr=0.001, num_epochs=500, step_size=500,
                save_path='models/trainable_gcesn_1l_mutag.pth',
                binary_classification=True, is_esn=True)

Total number of trainable parameters: 562


Fold 1/3
Fold 1 Results:
   Accuracy: 0.8571
   Average Sensitivity (Recall): 0.9512
   Average Specificity: 0.6818

Fold 2/3
Fold 2 Results:
   Accuracy: 0.7778
   Average Sensitivity (Recall): 0.8810
   Average Specificity: 0.5714

Fold 3/3
Fold 3 Results:
   Accuracy: 0.8226
   Average Sensitivity (Recall): 0.8333
   Average Specificity: 0.8000

Cross-Validation Results:
accuracy: Mean = 0.8192 ± 0.03
sensitivity: Mean = 0.8885 ± 0.05
specificity: Mean = 0.6844 ± 0.09
epochs: Mean = 413.3333 ± 60.42
training_times: Mean = 5.3099 ± 0.80
testing_times: Mean = 0.0041 ± 0.00
avg_memory_usage: Mean = 0.1120 ± 0.00
avg_gpu_usage: Mean = 0.0000 ± 0.00
max_memory_usage: Mean = 0.1133 ± 0.00
max_gpu_usage: Mean = 0.0000 ± 0.00


In [43]:
set_seed(42)
trainable_gcesn_mutag = TrainableGCESN_1layer(mutag_num_features, 2*mutag_num_features, mutag_num_classes, 
                        leaky_rate=0.8, num_iterations=3)
print(f"Total number of trainable parameters: {trainable_gcesn_mutag.count_parameters()}\n")
                
avg_metrics = single_train_test_cv(trainable_gcesn_mutag, mutag_dataset,
                lr=0.001, num_epochs=500, step_size=500,
                save_path='models/trainable_gcesn_1l_mutag.pth',
                binary_classification=True, is_esn=True)

Total number of trainable parameters: 562


Fold 1/3
Fold 1 Results:
   Accuracy: 0.8571
   Average Sensitivity (Recall): 0.9756
   Average Specificity: 0.6364

Fold 2/3
Fold 2 Results:
   Accuracy: 0.7460
   Average Sensitivity (Recall): 0.7857
   Average Specificity: 0.6667

Fold 3/3
Fold 3 Results:
   Accuracy: 0.8548
   Average Sensitivity (Recall): 0.9048
   Average Specificity: 0.7500

Cross-Validation Results:
accuracy: Mean = 0.8193 ± 0.05
sensitivity: Mean = 0.8887 ± 0.08
specificity: Mean = 0.6843 ± 0.05
epochs: Mean = 490.6667 ± 9.74
training_times: Mean = 6.1676 ± 0.22
testing_times: Mean = 0.0041 ± 0.00
avg_memory_usage: Mean = 0.1086 ± 0.00
avg_gpu_usage: Mean = 0.0000 ± 0.00
max_memory_usage: Mean = 0.1109 ± 0.00
max_gpu_usage: Mean = 0.0000 ± 0.00


In [44]:
set_seed(42)
trainable_gcesn_mutag = TrainableGCESN_1layer(mutag_num_features, 2*mutag_num_features, mutag_num_classes, 
                        leaky_rate=0.7, num_iterations=3)
print(f"Total number of trainable parameters: {trainable_gcesn_mutag.count_parameters()}\n")
                
avg_metrics = single_train_test_cv(trainable_gcesn_mutag, mutag_dataset,
                lr=0.001, num_epochs=500, step_size=500,
                save_path='models/trainable_gcesn_1l_mutag.pth',
                binary_classification=True, is_esn=True)

Total number of trainable parameters: 562


Fold 1/3
Fold 1 Results:
   Accuracy: 0.9206
   Average Sensitivity (Recall): 0.9756
   Average Specificity: 0.8182

Fold 2/3
Fold 2 Results:
   Accuracy: 0.7778
   Average Sensitivity (Recall): 0.8571
   Average Specificity: 0.6190

Fold 3/3
Fold 3 Results:
   Accuracy: 0.8871
   Average Sensitivity (Recall): 0.9286
   Average Specificity: 0.8000

Cross-Validation Results:
accuracy: Mean = 0.8618 ± 0.06
sensitivity: Mean = 0.9204 ± 0.05
specificity: Mean = 0.7457 ± 0.09
epochs: Mean = 487.6667 ± 16.05
training_times: Mean = 6.5285 ± 0.26
testing_times: Mean = 0.0050 ± 0.00
avg_memory_usage: Mean = 0.1029 ± 0.01
avg_gpu_usage: Mean = 0.0000 ± 0.00
max_memory_usage: Mean = 0.1085 ± 0.00
max_gpu_usage: Mean = 0.0000 ± 0.00


### Iteration 4

In [37]:
set_seed(42)
trainable_gcesn_mutag = TrainableGCESN_1layer(mutag_num_features, 2*mutag_num_features, mutag_num_classes, 
                        leaky_rate=1, num_iterations=4)
print(f"Total number of trainable parameters: {trainable_gcesn_mutag.count_parameters()}\n")
                
avg_metrics = single_train_test_cv(trainable_gcesn_mutag, mutag_dataset,
                lr=0.001, num_epochs=500, step_size=500,
                save_path='models/trainable_gcesn_1l_mutag.pth',
                binary_classification=True, is_esn=True)

Total number of trainable parameters: 562


Fold 1/3
Fold 1 Results:
   Accuracy: 0.7302
   Average Sensitivity (Recall): 1.0000
   Average Specificity: 0.2273

Fold 2/3
Fold 2 Results:
   Accuracy: 0.8095
   Average Sensitivity (Recall): 0.8333
   Average Specificity: 0.7619

Fold 3/3
Fold 3 Results:
   Accuracy: 0.7903
   Average Sensitivity (Recall): 0.8333
   Average Specificity: 0.7000

Cross-Validation Results:
accuracy: Mean = 0.7767 ± 0.03
sensitivity: Mean = 0.8889 ± 0.08
specificity: Mean = 0.5631 ± 0.24
epochs: Mean = 394.0000 ± 90.84
training_times: Mean = 5.5053 ± 1.24
testing_times: Mean = 0.0042 ± 0.00
avg_memory_usage: Mean = 0.1147 ± 0.00
avg_gpu_usage: Mean = 0.0000 ± 0.00
max_memory_usage: Mean = 0.1164 ± 0.00
max_gpu_usage: Mean = 0.0000 ± 0.00


In [38]:
set_seed(42)
trainable_gcesn_mutag = TrainableGCESN_1layer(mutag_num_features, 2*mutag_num_features, mutag_num_classes, 
                        leaky_rate=0.9, num_iterations=4)
print(f"Total number of trainable parameters: {trainable_gcesn_mutag.count_parameters()}\n")
                
avg_metrics = single_train_test_cv(trainable_gcesn_mutag, mutag_dataset,
                lr=0.001, num_epochs=500, step_size=500,
                save_path='models/trainable_gcesn_1l_mutag.pth',
                binary_classification=True, is_esn=True)

Total number of trainable parameters: 562


Fold 1/3
Fold 1 Results:
   Accuracy: 0.8730
   Average Sensitivity (Recall): 0.9756
   Average Specificity: 0.6818

Fold 2/3
Fold 2 Results:
   Accuracy: 0.7460
   Average Sensitivity (Recall): 0.8810
   Average Specificity: 0.4762

Fold 3/3
Fold 3 Results:
   Accuracy: 0.8065
   Average Sensitivity (Recall): 0.9762
   Average Specificity: 0.4500

Cross-Validation Results:
accuracy: Mean = 0.8085 ± 0.05
sensitivity: Mean = 0.9443 ± 0.04
specificity: Mean = 0.5360 ± 0.10
epochs: Mean = 280.3333 ± 28.76
training_times: Mean = 3.9070 ± 0.34
testing_times: Mean = 0.0039 ± 0.00
avg_memory_usage: Mean = 0.1873 ± 0.07
avg_gpu_usage: Mean = 0.0000 ± 0.00
max_memory_usage: Mean = 0.2255 ± 0.08
max_gpu_usage: Mean = 0.0000 ± 0.00


In [39]:
set_seed(42)
trainable_gcesn_mutag = TrainableGCESN_1layer(mutag_num_features, 2*mutag_num_features, mutag_num_classes, 
                        leaky_rate=0.8, num_iterations=4)
print(f"Total number of trainable parameters: {trainable_gcesn_mutag.count_parameters()}\n")
                
avg_metrics = single_train_test_cv(trainable_gcesn_mutag, mutag_dataset,
                lr=0.001, num_epochs=500, step_size=500,
                save_path='models/trainable_gcesn_1l_mutag.pth',
                binary_classification=True, is_esn=True)

Total number of trainable parameters: 562


Fold 1/3
Fold 1 Results:
   Accuracy: 0.8889
   Average Sensitivity (Recall): 1.0000
   Average Specificity: 0.6818

Fold 2/3
Fold 2 Results:
   Accuracy: 0.7619
   Average Sensitivity (Recall): 0.8095
   Average Specificity: 0.6667

Fold 3/3
Fold 3 Results:
   Accuracy: 0.8226
   Average Sensitivity (Recall): 0.9524
   Average Specificity: 0.5500

Cross-Validation Results:
accuracy: Mean = 0.8245 ± 0.05
sensitivity: Mean = 0.9206 ± 0.08
specificity: Mean = 0.6328 ± 0.06
epochs: Mean = 456.3333 ± 30.23
training_times: Mean = 6.2745 ± 0.42
testing_times: Mean = 0.0041 ± 0.00
avg_memory_usage: Mean = 0.2785 ± 0.00
avg_gpu_usage: Mean = 0.0000 ± 0.00
max_memory_usage: Mean = 0.2797 ± 0.00
max_gpu_usage: Mean = 0.0000 ± 0.00


In [40]:
set_seed(42)
trainable_gcesn_mutag = TrainableGCESN_1layer(mutag_num_features, 2*mutag_num_features, mutag_num_classes, 
                        leaky_rate=0.7, num_iterations=4)
print(f"Total number of trainable parameters: {trainable_gcesn_mutag.count_parameters()}\n")
                
avg_metrics = single_train_test_cv(trainable_gcesn_mutag, mutag_dataset,
                lr=0.001, num_epochs=500, step_size=500,
                save_path='models/trainable_gcesn_1l_mutag.pth',
                binary_classification=True, is_esn=True)

Total number of trainable parameters: 562


Fold 1/3
Fold 1 Results:
   Accuracy: 0.8889
   Average Sensitivity (Recall): 0.9756
   Average Specificity: 0.7273

Fold 2/3
Fold 2 Results:
   Accuracy: 0.7619
   Average Sensitivity (Recall): 0.8333
   Average Specificity: 0.6190

Fold 3/3
Fold 3 Results:
   Accuracy: 0.8387
   Average Sensitivity (Recall): 0.9286
   Average Specificity: 0.6500

Cross-Validation Results:
accuracy: Mean = 0.8298 ± 0.05
sensitivity: Mean = 0.9125 ± 0.06
specificity: Mean = 0.6654 ± 0.05
epochs: Mean = 416.6667 ± 76.07
training_times: Mean = 5.5680 ± 1.18
testing_times: Mean = 0.0040 ± 0.00
avg_memory_usage: Mean = 0.2782 ± 0.00
avg_gpu_usage: Mean = 0.0000 ± 0.00
max_memory_usage: Mean = 0.2786 ± 0.00
max_gpu_usage: Mean = 0.0000 ± 0.00


### Iteration 5

In [33]:
set_seed(42)
trainable_gcesn_mutag = TrainableGCESN_1layer(mutag_num_features, 2*mutag_num_features, mutag_num_classes, 
                        leaky_rate=1, num_iterations=5)
print(f"Total number of trainable parameters: {trainable_gcesn_mutag.count_parameters()}\n")
                
avg_metrics = single_train_test_cv(trainable_gcesn_mutag, mutag_dataset,
                lr=0.001, num_epochs=500, step_size=500,
                save_path='models/trainable_gcesn_1l_mutag.pth',
                binary_classification=True, is_esn=True)

Total number of trainable parameters: 562


Fold 1/3
Fold 1 Results:
   Accuracy: 0.7778
   Average Sensitivity (Recall): 1.0000
   Average Specificity: 0.3636

Fold 2/3
Fold 2 Results:
   Accuracy: 0.7778
   Average Sensitivity (Recall): 0.7619
   Average Specificity: 0.8095

Fold 3/3
Fold 3 Results:
   Accuracy: 0.8065
   Average Sensitivity (Recall): 0.7619
   Average Specificity: 0.9000

Cross-Validation Results:
accuracy: Mean = 0.7873 ± 0.01
sensitivity: Mean = 0.8413 ± 0.11
specificity: Mean = 0.6911 ± 0.23
epochs: Mean = 272.0000 ± 57.31
training_times: Mean = 4.0282 ± 0.90
testing_times: Mean = 0.0045 ± 0.00
avg_memory_usage: Mean = 0.1145 ± 0.00
avg_gpu_usage: Mean = 0.0000 ± 0.00
max_memory_usage: Mean = 0.1151 ± 0.00
max_gpu_usage: Mean = 0.0000 ± 0.00


In [34]:
set_seed(42)
trainable_gcesn_mutag = TrainableGCESN_1layer(mutag_num_features, 2*mutag_num_features, mutag_num_classes, 
                        leaky_rate=0.9, num_iterations=5)
print(f"Total number of trainable parameters: {trainable_gcesn_mutag.count_parameters()}\n")
                
avg_metrics = single_train_test_cv(trainable_gcesn_mutag, mutag_dataset,
                lr=0.001, num_epochs=500, step_size=500,
                save_path='models/trainable_gcesn_1l_mutag.pth',
                binary_classification=True, is_esn=True)

Total number of trainable parameters: 562


Fold 1/3
Fold 1 Results:
   Accuracy: 0.8730
   Average Sensitivity (Recall): 1.0000
   Average Specificity: 0.6364

Fold 2/3
Fold 2 Results:
   Accuracy: 0.7937
   Average Sensitivity (Recall): 0.7857
   Average Specificity: 0.8095

Fold 3/3
Fold 3 Results:
   Accuracy: 0.7903
   Average Sensitivity (Recall): 0.8095
   Average Specificity: 0.7500

Cross-Validation Results:
accuracy: Mean = 0.8190 ± 0.04
sensitivity: Mean = 0.8651 ± 0.10
specificity: Mean = 0.7320 ± 0.07
epochs: Mean = 263.3333 ± 64.77
training_times: Mean = 3.8859 ± 0.95
testing_times: Mean = 0.0045 ± 0.00
avg_memory_usage: Mean = 0.1175 ± 0.00
avg_gpu_usage: Mean = 0.0000 ± 0.00
max_memory_usage: Mean = 0.1179 ± 0.00
max_gpu_usage: Mean = 0.0000 ± 0.00


In [35]:
set_seed(42)
trainable_gcesn_mutag = TrainableGCESN_1layer(mutag_num_features, 2*mutag_num_features, mutag_num_classes, 
                        leaky_rate=0.8, num_iterations=5)
print(f"Total number of trainable parameters: {trainable_gcesn_mutag.count_parameters()}\n")
                
avg_metrics = single_train_test_cv(trainable_gcesn_mutag, mutag_dataset,
                lr=0.001, num_epochs=500, step_size=500,
                save_path='models/trainable_gcesn_1l_mutag.pth',
                binary_classification=True, is_esn=True)

Total number of trainable parameters: 562


Fold 1/3
Fold 1 Results:
   Accuracy: 0.7302
   Average Sensitivity (Recall): 1.0000
   Average Specificity: 0.2273

Fold 2/3
Fold 2 Results:
   Accuracy: 0.7778
   Average Sensitivity (Recall): 0.7857
   Average Specificity: 0.7619

Fold 3/3
Fold 3 Results:
   Accuracy: 0.8548
   Average Sensitivity (Recall): 0.9286
   Average Specificity: 0.7000

Cross-Validation Results:
accuracy: Mean = 0.7876 ± 0.05
sensitivity: Mean = 0.9048 ± 0.09
specificity: Mean = 0.5631 ± 0.24
epochs: Mean = 338.0000 ± 184.41
training_times: Mean = 4.9667 ± 2.69
testing_times: Mean = 0.0045 ± 0.00
avg_memory_usage: Mean = 0.1184 ± 0.00
avg_gpu_usage: Mean = 0.0000 ± 0.00
max_memory_usage: Mean = 0.1186 ± 0.00
max_gpu_usage: Mean = 0.0000 ± 0.00


In [36]:
set_seed(42)
trainable_gcesn_mutag_2 = TrainableGCESN_2layer(mutag_num_features, 2*mutag_num_features, mutag_num_classes,
                        leaky_rate=0.7, num_iterations=5)
print(f"Total number of trainable parameters: {trainable_gcesn_mutag_2.count_parameters()}\n")
                
avg_metrics = single_train_test_cv(trainable_gcesn_mutag_2, mutag_dataset,
                lr=0.005, num_epochs=500, step_size=500,  
                save_path='models/trainable_gcesn_2l_mutag.pth',
                binary_classification=True, is_esn=True)

Total number of trainable parameters: 1962


Fold 1/3
Fold 1 Results:
   Accuracy: 0.9365
   Average Sensitivity (Recall): 0.9024
   Average Specificity: 1.0000

Fold 2/3
Fold 2 Results:
   Accuracy: 0.6984
   Average Sensitivity (Recall): 0.6905
   Average Specificity: 0.7143

Fold 3/3
Fold 3 Results:
   Accuracy: 0.6452
   Average Sensitivity (Recall): 0.4762
   Average Specificity: 1.0000

Cross-Validation Results:
accuracy: Mean = 0.7600 ± 0.13
sensitivity: Mean = 0.6897 ± 0.17
specificity: Mean = 0.9048 ± 0.13
epochs: Mean = 68.0000 ± 31.82
training_times: Mean = 1.6105 ± 0.71
testing_times: Mean = 0.0061 ± 0.00
avg_memory_usage: Mean = 0.1184 ± 0.00
avg_gpu_usage: Mean = 0.0000 ± 0.00
max_memory_usage: Mean = 0.1194 ± 0.00
max_gpu_usage: Mean = 0.0000 ± 0.00


# EMCI-AD

## GCESN

### Iteration 1

In [71]:
set_seed(42)
gcesn_emci = GCESN_1layer(emci_num_features, 2*emci_num_features, emci_num_classes, leaky_rate=1, num_iterations=1)
print(gcesn_emci)
print(f"Total number of trainable parameters: {gcesn_emci.count_parameters()}\n")

avg_metrics = multi_train_test_cv(gcesn_emci, emci_dataset,
                lr=0.001, num_epochs=500, step_size=500, gamma=0.5, 
                num_runs=3, binary_classification=False, 
                best_model_path='models/best_gcesn_1l_emci.pth')


GCESN_1layer(
  (fc1): Linear(in_features=16, out_features=16, bias=True)
  (bn): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc): Linear(in_features=16, out_features=2, bias=True)
)
Total number of trainable parameters: 338


Run 1/3
Fold 1/3
Fold 1 Results:
   Accuracy: 0.4444
   Average Sensitivity (Recall): 0.4417
   Average Specificity: 0.4417
Fold 2/3
Fold 2 Results:
   Accuracy: 0.4444
   Average Sensitivity (Recall): 0.5000
   Average Specificity: 0.5000
Fold 3/3
Fold 3 Results:
   Accuracy: 0.3409
   Average Sensitivity (Recall): 0.3505
   Average Specificity: 0.3505
Run 1 Cross-Validation Results:
accuracy: Mean = 0.4099, Std = 0.0488
sensitivity: Mean = 0.4307, Std = 0.0615
specificity: Mean = 0.4307, Std = 0.0615

Run 2/3
Fold 1/3
Fold 1 Results:
   Accuracy: 0.4667
   Average Sensitivity (Recall): 0.4773
   Average Specificity: 0.4773
Fold 2/3
Fold 2 Results:
   Accuracy: 0.4667
   Average Sensitivity (Recall): 0.4900
   Average Speci

In [72]:
set_seed(42)
gcesn_emci = GCESN_1layer(emci_num_features, 2*emci_num_features, emci_num_classes, leaky_rate=0.9, num_iterations=1)
print(gcesn_emci)
print(f"Total number of trainable parameters: {gcesn_emci.count_parameters()}\n")

avg_metrics = multi_train_test_cv(gcesn_emci, emci_dataset,
                lr=0.001, num_epochs=500, step_size=500, gamma=0.5, 
                num_runs=3, binary_classification=False, 
                best_model_path='models/best_gcesn_1l_emci.pth')


GCESN_1layer(
  (fc1): Linear(in_features=16, out_features=16, bias=True)
  (bn): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc): Linear(in_features=16, out_features=2, bias=True)
)
Total number of trainable parameters: 338


Run 1/3
Fold 1/3
Fold 1 Results:
   Accuracy: 0.4444
   Average Sensitivity (Recall): 0.4516
   Average Specificity: 0.4516
Fold 2/3
Fold 2 Results:
   Accuracy: 0.3778
   Average Sensitivity (Recall): 0.4150
   Average Specificity: 0.4150
Fold 3/3
Fold 3 Results:
   Accuracy: 0.4318
   Average Sensitivity (Recall): 0.5000
   Average Specificity: 0.5000
Run 1 Cross-Validation Results:
accuracy: Mean = 0.4180, Std = 0.0289
sensitivity: Mean = 0.4555, Std = 0.0348
specificity: Mean = 0.4555, Std = 0.0348

Run 2/3
Fold 1/3
Fold 1 Results:
   Accuracy: 0.4000
   Average Sensitivity (Recall): 0.4042
   Average Specificity: 0.4042
Fold 2/3
Fold 2 Results:
   Accuracy: 0.4667
   Average Sensitivity (Recall): 0.4650
   Average Speci

In [73]:
set_seed(42)
gcesn_emci = GCESN_1layer(emci_num_features, 2*emci_num_features, emci_num_classes, leaky_rate=0.8, num_iterations=1)
print(gcesn_emci)
print(f"Total number of trainable parameters: {gcesn_emci.count_parameters()}\n")

avg_metrics = multi_train_test_cv(gcesn_emci, emci_dataset,
                lr=0.001, num_epochs=500, step_size=500, gamma=0.5, 
                num_runs=3, binary_classification=False, 
                best_model_path='models/best_gcesn_1l_emci.pth')


GCESN_1layer(
  (fc1): Linear(in_features=16, out_features=16, bias=True)
  (bn): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc): Linear(in_features=16, out_features=2, bias=True)
)
Total number of trainable parameters: 338


Run 1/3
Fold 1/3
Fold 1 Results:
   Accuracy: 0.4667
   Average Sensitivity (Recall): 0.4664
   Average Specificity: 0.4664
Fold 2/3
Fold 2 Results:
   Accuracy: 0.4889
   Average Sensitivity (Recall): 0.5200
   Average Specificity: 0.5200
Fold 3/3
Fold 3 Results:
   Accuracy: 0.3864
   Average Sensitivity (Recall): 0.4032
   Average Specificity: 0.4032
Run 1 Cross-Validation Results:
accuracy: Mean = 0.4473, Std = 0.0440
sensitivity: Mean = 0.4632, Std = 0.0478
specificity: Mean = 0.4632, Std = 0.0478

Run 2/3
Fold 1/3
Fold 1 Results:
   Accuracy: 0.4444
   Average Sensitivity (Recall): 0.4486
   Average Specificity: 0.4486
Fold 2/3
Fold 2 Results:
   Accuracy: 0.4444
   Average Sensitivity (Recall): 0.4950
   Average Speci

In [75]:
set_seed(42)
gcesn_emci = GCESN_1layer(emci_num_features, 2*emci_num_features, emci_num_classes, leaky_rate=0.7, num_iterations=1)
print(gcesn_emci)
print(f"Total number of trainable parameters: {gcesn_emci.count_parameters()}\n")

avg_metrics = multi_train_test_cv(gcesn_emci, emci_dataset,
                lr=0.001, num_epochs=500, step_size=500, gamma=0.5, 
                num_runs=3, binary_classification=False, 
                best_model_path='models/best_gcesn_1l_emci.pth')

GCESN_1layer(
  (fc1): Linear(in_features=16, out_features=16, bias=True)
  (bn): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc): Linear(in_features=16, out_features=2, bias=True)
)
Total number of trainable parameters: 338


Run 1/3
Fold 1/3
Fold 1 Results:
   Accuracy: 0.4444
   Average Sensitivity (Recall): 0.4397
   Average Specificity: 0.4397
Fold 2/3
Fold 2 Results:
   Accuracy: 0.4000
   Average Sensitivity (Recall): 0.4500
   Average Specificity: 0.4500
Fold 3/3
Fold 3 Results:
   Accuracy: 0.4091
   Average Sensitivity (Recall): 0.4484
   Average Specificity: 0.4484
Run 1 Cross-Validation Results:
accuracy: Mean = 0.4178, Std = 0.0192
sensitivity: Mean = 0.4460, Std = 0.0045
specificity: Mean = 0.4460, Std = 0.0045

Run 2/3
Fold 1/3
Fold 1 Results:
   Accuracy: 0.4000
   Average Sensitivity (Recall): 0.4032
   Average Specificity: 0.4032
Fold 2/3
Fold 2 Results:
   Accuracy: 0.4667
   Average Sensitivity (Recall): 0.4550
   Average Speci

### Iteration 2

In [76]:
set_seed(42)
gcesn_emci = GCESN_1layer(emci_num_features, 2*emci_num_features, emci_num_classes, leaky_rate=1, num_iterations=2)
print(gcesn_emci)
print(f"Total number of trainable parameters: {gcesn_emci.count_parameters()}\n")

avg_metrics = multi_train_test_cv(gcesn_emci, emci_dataset,
                lr=0.001, num_epochs=500, step_size=500, gamma=0.5, 
                num_runs=3, binary_classification=False, 
                best_model_path='models/best_gcesn_1l_emci.pth')


GCESN_1layer(
  (fc1): Linear(in_features=16, out_features=16, bias=True)
  (bn): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc): Linear(in_features=16, out_features=2, bias=True)
)
Total number of trainable parameters: 338


Run 1/3
Fold 1/3
Fold 1 Results:
   Accuracy: 0.4667
   Average Sensitivity (Recall): 0.4763
   Average Specificity: 0.4763
Fold 2/3
Fold 2 Results:
   Accuracy: 0.5333
   Average Sensitivity (Recall): 0.5200
   Average Specificity: 0.5200
Fold 3/3
Fold 3 Results:
   Accuracy: 0.5227
   Average Sensitivity (Recall): 0.5421
   Average Specificity: 0.5421
Run 1 Cross-Validation Results:
accuracy: Mean = 0.5076, Std = 0.0292
sensitivity: Mean = 0.5128, Std = 0.0273
specificity: Mean = 0.5128, Std = 0.0273

Run 2/3
Fold 1/3
Fold 1 Results:
   Accuracy: 0.6000
   Average Sensitivity (Recall): 0.5988
   Average Specificity: 0.5988
Fold 2/3
Fold 2 Results:
   Accuracy: 0.5778
   Average Sensitivity (Recall): 0.5600
   Average Speci

In [77]:
set_seed(42)
gcesn_emci = GCESN_1layer(emci_num_features, 2*emci_num_features, emci_num_classes, leaky_rate=0.9, num_iterations=2)
print(gcesn_emci)
print(f"Total number of trainable parameters: {gcesn_emci.count_parameters()}\n")

avg_metrics = multi_train_test_cv(gcesn_emci, emci_dataset,
                lr=0.001, num_epochs=500, step_size=500, gamma=0.5, 
                num_runs=3, binary_classification=False, 
                best_model_path='models/best_gcesn_1l_emci.pth')


GCESN_1layer(
  (fc1): Linear(in_features=16, out_features=16, bias=True)
  (bn): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc): Linear(in_features=16, out_features=2, bias=True)
)
Total number of trainable parameters: 338


Run 1/3
Fold 1/3
Fold 1 Results:
   Accuracy: 0.4889
   Average Sensitivity (Recall): 0.4911
   Average Specificity: 0.4911
Fold 2/3
Fold 2 Results:
   Accuracy: 0.5333
   Average Sensitivity (Recall): 0.5550
   Average Specificity: 0.5550
Fold 3/3
Fold 3 Results:
   Accuracy: 0.5000
   Average Sensitivity (Recall): 0.5158
   Average Specificity: 0.5158
Run 1 Cross-Validation Results:
accuracy: Mean = 0.5074, Std = 0.0189
sensitivity: Mean = 0.5206, Std = 0.0263
specificity: Mean = 0.5206, Std = 0.0263

Run 2/3
Fold 1/3
Fold 1 Results:
   Accuracy: 0.4889
   Average Sensitivity (Recall): 0.4951
   Average Specificity: 0.4951
Fold 2/3
Fold 2 Results:
   Accuracy: 0.5111
   Average Sensitivity (Recall): 0.5300
   Average Speci

In [78]:
set_seed(42)
gcesn_emci = GCESN_1layer(emci_num_features, 2*emci_num_features, emci_num_classes, leaky_rate=0.8, num_iterations=2)
print(gcesn_emci)
print(f"Total number of trainable parameters: {gcesn_emci.count_parameters()}\n")

avg_metrics = multi_train_test_cv(gcesn_emci, emci_dataset,
                lr=0.001, num_epochs=500, step_size=500, gamma=0.5, 
                num_runs=3, binary_classification=False, 
                best_model_path='models/best_gcesn_1l_emci.pth')


GCESN_1layer(
  (fc1): Linear(in_features=16, out_features=16, bias=True)
  (bn): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc): Linear(in_features=16, out_features=2, bias=True)
)
Total number of trainable parameters: 338


Run 1/3
Fold 1/3
Fold 1 Results:
   Accuracy: 0.4667
   Average Sensitivity (Recall): 0.4733
   Average Specificity: 0.4733
Fold 2/3
Fold 2 Results:
   Accuracy: 0.4889
   Average Sensitivity (Recall): 0.5100
   Average Specificity: 0.5100
Fold 3/3
Fold 3 Results:
   Accuracy: 0.4091
   Average Sensitivity (Recall): 0.4358
   Average Specificity: 0.4358
Run 1 Cross-Validation Results:
accuracy: Mean = 0.4549, Std = 0.0336
sensitivity: Mean = 0.4730, Std = 0.0303
specificity: Mean = 0.4730, Std = 0.0303

Run 2/3
Fold 1/3
Fold 1 Results:
   Accuracy: 0.5111
   Average Sensitivity (Recall): 0.5128
   Average Specificity: 0.5128
Fold 2/3
Fold 2 Results:
   Accuracy: 0.5111
   Average Sensitivity (Recall): 0.5050
   Average Speci

In [79]:
set_seed(42)
gcesn_emci = GCESN_1layer(emci_num_features, 2*emci_num_features, emci_num_classes, leaky_rate=0.7, num_iterations=2)
print(gcesn_emci)
print(f"Total number of trainable parameters: {gcesn_emci.count_parameters()}\n")

avg_metrics = multi_train_test_cv(gcesn_emci, emci_dataset,
                lr=0.001, num_epochs=500, step_size=500, gamma=0.5, 
                num_runs=3, binary_classification=False, 
                best_model_path='models/best_gcesn_1l_emci.pth')


GCESN_1layer(
  (fc1): Linear(in_features=16, out_features=16, bias=True)
  (bn): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc): Linear(in_features=16, out_features=2, bias=True)
)
Total number of trainable parameters: 338


Run 1/3
Fold 1/3
Fold 1 Results:
   Accuracy: 0.5111
   Average Sensitivity (Recall): 0.5119
   Average Specificity: 0.5119
Fold 2/3
Fold 2 Results:
   Accuracy: 0.5778
   Average Sensitivity (Recall): 0.6000
   Average Specificity: 0.6000
Fold 3/3
Fold 3 Results:
   Accuracy: 0.4091
   Average Sensitivity (Recall): 0.4484
   Average Specificity: 0.4484
Run 1 Cross-Validation Results:
accuracy: Mean = 0.4993, Std = 0.0694
sensitivity: Mean = 0.5201, Std = 0.0622
specificity: Mean = 0.5201, Std = 0.0622

Run 2/3
Fold 1/3
Fold 1 Results:
   Accuracy: 0.4667
   Average Sensitivity (Recall): 0.4713
   Average Specificity: 0.4713
Fold 2/3
Fold 2 Results:
   Accuracy: 0.4000
   Average Sensitivity (Recall): 0.4300
   Average Speci

### Iteration 3

In [80]:
set_seed(42)
gcesn_emci = GCESN_1layer(emci_num_features, 2*emci_num_features, emci_num_classes, leaky_rate=1, num_iterations=3)
print(gcesn_emci)
print(f"Total number of trainable parameters: {gcesn_emci.count_parameters()}\n")

avg_metrics = multi_train_test_cv(gcesn_emci, emci_dataset,
                lr=0.001, num_epochs=500, step_size=500, gamma=0.5, 
                num_runs=3, binary_classification=False, 
                best_model_path='models/best_gcesn_1l_emci.pth')


GCESN_1layer(
  (fc1): Linear(in_features=16, out_features=16, bias=True)
  (bn): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc): Linear(in_features=16, out_features=2, bias=True)
)
Total number of trainable parameters: 338


Run 1/3
Fold 1/3
Fold 1 Results:
   Accuracy: 0.4222
   Average Sensitivity (Recall): 0.4259
   Average Specificity: 0.4259
Fold 2/3
Fold 2 Results:
   Accuracy: 0.4667
   Average Sensitivity (Recall): 0.4950
   Average Specificity: 0.4950
Fold 3/3
Fold 3 Results:
   Accuracy: 0.4545
   Average Sensitivity (Recall): 0.4758
   Average Specificity: 0.4758
Run 1 Cross-Validation Results:
accuracy: Mean = 0.4478, Std = 0.0188
sensitivity: Mean = 0.4656, Std = 0.0291
specificity: Mean = 0.4656, Std = 0.0291

Run 2/3
Fold 1/3
Fold 1 Results:
   Accuracy: 0.5556
   Average Sensitivity (Recall): 0.5603
   Average Specificity: 0.5603
Fold 2/3
Fold 2 Results:
   Accuracy: 0.4000
   Average Sensitivity (Recall): 0.4100
   Average Speci

In [81]:
set_seed(42)
gcesn_emci = GCESN_1layer(emci_num_features, 2*emci_num_features, emci_num_classes, leaky_rate=0.9, num_iterations=3)
print(gcesn_emci)
print(f"Total number of trainable parameters: {gcesn_emci.count_parameters()}\n")

avg_metrics = multi_train_test_cv(gcesn_emci, emci_dataset,
                lr=0.001, num_epochs=500, step_size=500, gamma=0.5, 
                num_runs=3, binary_classification=False, 
                best_model_path='models/best_gcesn_1l_emci.pth')



GCESN_1layer(
  (fc1): Linear(in_features=16, out_features=16, bias=True)
  (bn): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc): Linear(in_features=16, out_features=2, bias=True)
)
Total number of trainable parameters: 338


Run 1/3
Fold 1/3
Fold 1 Results:
   Accuracy: 0.4222
   Average Sensitivity (Recall): 0.4249
   Average Specificity: 0.4249
Fold 2/3
Fold 2 Results:
   Accuracy: 0.5333
   Average Sensitivity (Recall): 0.5750
   Average Specificity: 0.5750
Fold 3/3
Fold 3 Results:
   Accuracy: 0.4091
   Average Sensitivity (Recall): 0.4484
   Average Specificity: 0.4484
Run 1 Cross-Validation Results:
accuracy: Mean = 0.4549, Std = 0.0557
sensitivity: Mean = 0.4828, Std = 0.0659
specificity: Mean = 0.4828, Std = 0.0659

Run 2/3
Fold 1/3
Fold 1 Results:
   Accuracy: 0.6000
   Average Sensitivity (Recall): 0.5998
   Average Specificity: 0.5998
Fold 2/3
Fold 2 Results:
   Accuracy: 0.4667
   Average Sensitivity (Recall): 0.4800
   Average Speci

In [82]:
set_seed(42)
gcesn_emci = GCESN_1layer(emci_num_features, 2*emci_num_features, emci_num_classes, leaky_rate=0.8, num_iterations=3)
print(gcesn_emci)
print(f"Total number of trainable parameters: {gcesn_emci.count_parameters()}\n")

avg_metrics = multi_train_test_cv(gcesn_emci, emci_dataset,
                lr=0.001, num_epochs=500, step_size=500, gamma=0.5, 
                num_runs=3, binary_classification=False, 
                best_model_path='models/best_gcesn_1l_emci.pth')


GCESN_1layer(
  (fc1): Linear(in_features=16, out_features=16, bias=True)
  (bn): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc): Linear(in_features=16, out_features=2, bias=True)
)
Total number of trainable parameters: 338


Run 1/3
Fold 1/3
Fold 1 Results:
   Accuracy: 0.4889
   Average Sensitivity (Recall): 0.4921
   Average Specificity: 0.4921
Fold 2/3
Fold 2 Results:
   Accuracy: 0.4444
   Average Sensitivity (Recall): 0.4550
   Average Specificity: 0.4550
Fold 3/3
Fold 3 Results:
   Accuracy: 0.4091
   Average Sensitivity (Recall): 0.4358
   Average Specificity: 0.4358
Run 1 Cross-Validation Results:
accuracy: Mean = 0.4475, Std = 0.0326
sensitivity: Mean = 0.4610, Std = 0.0234
specificity: Mean = 0.4610, Std = 0.0234

Run 2/3
Fold 1/3
Fold 1 Results:
   Accuracy: 0.4889
   Average Sensitivity (Recall): 0.4921
   Average Specificity: 0.4921
Fold 2/3
Fold 2 Results:
   Accuracy: 0.4667
   Average Sensitivity (Recall): 0.4850
   Average Speci

In [83]:
set_seed(42)
gcesn_emci = GCESN_1layer(emci_num_features, 2*emci_num_features, emci_num_classes, leaky_rate=0.7, num_iterations=3)
print(gcesn_emci)
print(f"Total number of trainable parameters: {gcesn_emci.count_parameters()}\n")

avg_metrics = multi_train_test_cv(gcesn_emci, emci_dataset,
                lr=0.001, num_epochs=500, step_size=500, gamma=0.5, 
                num_runs=3, binary_classification=False, 
                best_model_path='models/best_gcesn_1l_emci.pth')


GCESN_1layer(
  (fc1): Linear(in_features=16, out_features=16, bias=True)
  (bn): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc): Linear(in_features=16, out_features=2, bias=True)
)
Total number of trainable parameters: 338


Run 1/3
Fold 1/3
Fold 1 Results:
   Accuracy: 0.5778
   Average Sensitivity (Recall): 0.5820
   Average Specificity: 0.5820
Fold 2/3
Fold 2 Results:
   Accuracy: 0.5111
   Average Sensitivity (Recall): 0.5050
   Average Specificity: 0.5050
Fold 3/3
Fold 3 Results:
   Accuracy: 0.3636
   Average Sensitivity (Recall): 0.3895
   Average Specificity: 0.3895
Run 1 Cross-Validation Results:
accuracy: Mean = 0.4842, Std = 0.0895
sensitivity: Mean = 0.4922, Std = 0.0791
specificity: Mean = 0.4922, Std = 0.0791

Run 2/3
Fold 1/3
Fold 1 Results:
   Accuracy: 0.5111
   Average Sensitivity (Recall): 0.5168
   Average Specificity: 0.5168
Fold 2/3
Fold 2 Results:
   Accuracy: 0.4889
   Average Sensitivity (Recall): 0.5150
   Average Speci

### Iteration 4

In [84]:
set_seed(42)
gcesn_emci = GCESN_1layer(emci_num_features, 2*emci_num_features, emci_num_classes, leaky_rate=1, num_iterations=4)
print(gcesn_emci)
print(f"Total number of trainable parameters: {gcesn_emci.count_parameters()}\n")

avg_metrics = multi_train_test_cv(gcesn_emci, emci_dataset,
                lr=0.001, num_epochs=500, step_size=500, gamma=0.5, 
                num_runs=3, binary_classification=False, 
                best_model_path='models/best_gcesn_1l_emci.pth')


GCESN_1layer(
  (fc1): Linear(in_features=16, out_features=16, bias=True)
  (bn): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc): Linear(in_features=16, out_features=2, bias=True)
)
Total number of trainable parameters: 338


Run 1/3
Fold 1/3
Fold 1 Results:
   Accuracy: 0.5778
   Average Sensitivity (Recall): 0.5791
   Average Specificity: 0.5791
Fold 2/3
Fold 2 Results:
   Accuracy: 0.4222
   Average Sensitivity (Recall): 0.4300
   Average Specificity: 0.4300
Fold 3/3
Fold 3 Results:
   Accuracy: 0.3864
   Average Sensitivity (Recall): 0.4032
   Average Specificity: 0.4032
Run 1 Cross-Validation Results:
accuracy: Mean = 0.4621, Std = 0.0831
sensitivity: Mean = 0.4707, Std = 0.0774
specificity: Mean = 0.4707, Std = 0.0774

Run 2/3
Fold 1/3
Fold 1 Results:
   Accuracy: 0.5778
   Average Sensitivity (Recall): 0.5741
   Average Specificity: 0.5741
Fold 2/3
Fold 2 Results:
   Accuracy: 0.5333
   Average Sensitivity (Recall): 0.5250
   Average Speci

In [85]:
set_seed(42)
gcesn_emci = GCESN_1layer(emci_num_features, 2*emci_num_features, emci_num_classes, leaky_rate=0.9, num_iterations=4)
print(gcesn_emci)
print(f"Total number of trainable parameters: {gcesn_emci.count_parameters()}\n")

avg_metrics = multi_train_test_cv(gcesn_emci, emci_dataset,
                lr=0.001, num_epochs=500, step_size=500, gamma=0.5, 
                num_runs=3, binary_classification=False, 
                best_model_path='models/best_gcesn_1l_emci.pth')


GCESN_1layer(
  (fc1): Linear(in_features=16, out_features=16, bias=True)
  (bn): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc): Linear(in_features=16, out_features=2, bias=True)
)
Total number of trainable parameters: 338


Run 1/3
Fold 1/3
Fold 1 Results:
   Accuracy: 0.4667
   Average Sensitivity (Recall): 0.4713
   Average Specificity: 0.4713
Fold 2/3
Fold 2 Results:
   Accuracy: 0.5556
   Average Sensitivity (Recall): 0.5400
   Average Specificity: 0.5400
Fold 3/3
Fold 3 Results:
   Accuracy: 0.4545
   Average Sensitivity (Recall): 0.4758
   Average Specificity: 0.4758
Run 1 Cross-Validation Results:
accuracy: Mean = 0.4923, Std = 0.0450
sensitivity: Mean = 0.4957, Std = 0.0314
specificity: Mean = 0.4957, Std = 0.0314

Run 2/3
Fold 1/3
Fold 1 Results:
   Accuracy: 0.5333
   Average Sensitivity (Recall): 0.5316
   Average Specificity: 0.5316
Fold 2/3
Fold 2 Results:
   Accuracy: 0.4667
   Average Sensitivity (Recall): 0.4800
   Average Speci

In [86]:
set_seed(42)
gcesn_emci = GCESN_1layer(emci_num_features, 2*emci_num_features, emci_num_classes, leaky_rate=0.8, num_iterations=4)
print(gcesn_emci)
print(f"Total number of trainable parameters: {gcesn_emci.count_parameters()}\n")

avg_metrics = multi_train_test_cv(gcesn_emci, emci_dataset,
                lr=0.001, num_epochs=500, step_size=500, gamma=0.5, 
                num_runs=3, binary_classification=False, 
                best_model_path='models/best_gcesn_1l_emci.pth')


GCESN_1layer(
  (fc1): Linear(in_features=16, out_features=16, bias=True)
  (bn): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc): Linear(in_features=16, out_features=2, bias=True)
)
Total number of trainable parameters: 338


Run 1/3
Fold 1/3
Fold 1 Results:
   Accuracy: 0.4667
   Average Sensitivity (Recall): 0.4733
   Average Specificity: 0.4733
Fold 2/3
Fold 2 Results:
   Accuracy: 0.4667
   Average Sensitivity (Recall): 0.4750
   Average Specificity: 0.4750
Fold 3/3
Fold 3 Results:
   Accuracy: 0.5000
   Average Sensitivity (Recall): 0.5284
   Average Specificity: 0.5284
Run 1 Cross-Validation Results:
accuracy: Mean = 0.4778, Std = 0.0157
sensitivity: Mean = 0.4922, Std = 0.0256
specificity: Mean = 0.4922, Std = 0.0256

Run 2/3
Fold 1/3
Fold 1 Results:
   Accuracy: 0.5556
   Average Sensitivity (Recall): 0.5563
   Average Specificity: 0.5563
Fold 2/3
Fold 2 Results:
   Accuracy: 0.4889
   Average Sensitivity (Recall): 0.5050
   Average Speci

In [87]:
set_seed(42)
gcesn_emci = GCESN_1layer(emci_num_features, 2*emci_num_features, emci_num_classes, leaky_rate=0.7, num_iterations=4)
print(gcesn_emci)
print(f"Total number of trainable parameters: {gcesn_emci.count_parameters()}\n")

avg_metrics = multi_train_test_cv(gcesn_emci, emci_dataset,
                lr=0.001, num_epochs=500, step_size=500, gamma=0.5, 
                num_runs=3, binary_classification=False, 
                best_model_path='models/best_gcesn_1l_emci.pth')


GCESN_1layer(
  (fc1): Linear(in_features=16, out_features=16, bias=True)
  (bn): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc): Linear(in_features=16, out_features=2, bias=True)
)
Total number of trainable parameters: 338


Run 1/3
Fold 1/3
Fold 1 Results:
   Accuracy: 0.5333
   Average Sensitivity (Recall): 0.5346
   Average Specificity: 0.5346
Fold 2/3
Fold 2 Results:
   Accuracy: 0.4889
   Average Sensitivity (Recall): 0.5100
   Average Specificity: 0.5100
Fold 3/3
Fold 3 Results:
   Accuracy: 0.6136
   Average Sensitivity (Recall): 0.6284
   Average Specificity: 0.6284
Run 1 Cross-Validation Results:
accuracy: Mean = 0.5453, Std = 0.0516
sensitivity: Mean = 0.5577, Std = 0.0510
specificity: Mean = 0.5577, Std = 0.0510

Run 2/3
Fold 1/3
Fold 1 Results:
   Accuracy: 0.5333
   Average Sensitivity (Recall): 0.5296
   Average Specificity: 0.5296
Fold 2/3
Fold 2 Results:
   Accuracy: 0.4889
   Average Sensitivity (Recall): 0.5050
   Average Speci

### Iteration 5

In [88]:
set_seed(42)
gcesn_emci = GCESN_1layer(emci_num_features, 2*emci_num_features, emci_num_classes, leaky_rate=1, num_iterations=5)
print(gcesn_emci)
print(f"Total number of trainable parameters: {gcesn_emci.count_parameters()}\n")

avg_metrics = multi_train_test_cv(gcesn_emci, emci_dataset,
                lr=0.001, num_epochs=500, step_size=500, gamma=0.5, 
                num_runs=3, binary_classification=False, 
                best_model_path='models/best_gcesn_1l_emci.pth')

GCESN_1layer(
  (fc1): Linear(in_features=16, out_features=16, bias=True)
  (bn): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc): Linear(in_features=16, out_features=2, bias=True)
)
Total number of trainable parameters: 338


Run 1/3
Fold 1/3
Fold 1 Results:
   Accuracy: 0.5778
   Average Sensitivity (Recall): 0.5761
   Average Specificity: 0.5761
Fold 2/3
Fold 2 Results:
   Accuracy: 0.4444
   Average Sensitivity (Recall): 0.4300
   Average Specificity: 0.4300
Fold 3/3
Fold 3 Results:
   Accuracy: 0.4091
   Average Sensitivity (Recall): 0.4358
   Average Specificity: 0.4358
Run 1 Cross-Validation Results:
accuracy: Mean = 0.4771, Std = 0.0726
sensitivity: Mean = 0.4806, Std = 0.0675
specificity: Mean = 0.4806, Std = 0.0675

Run 2/3
Fold 1/3
Fold 1 Results:
   Accuracy: 0.4667
   Average Sensitivity (Recall): 0.4723
   Average Specificity: 0.4723
Fold 2/3
Fold 2 Results:
   Accuracy: 0.6000
   Average Sensitivity (Recall): 0.5950
   Average Speci

In [89]:
set_seed(42)
gcesn_emci = GCESN_1layer(emci_num_features, 2*emci_num_features, emci_num_classes, leaky_rate=0.9, num_iterations=5)
print(gcesn_emci)
print(f"Total number of trainable parameters: {gcesn_emci.count_parameters()}\n")

avg_metrics = multi_train_test_cv(gcesn_emci, emci_dataset,
                lr=0.001, num_epochs=500, step_size=500, gamma=0.5, 
                num_runs=3, binary_classification=False, 
                best_model_path='models/best_gcesn_1l_emci.pth')


GCESN_1layer(
  (fc1): Linear(in_features=16, out_features=16, bias=True)
  (bn): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc): Linear(in_features=16, out_features=2, bias=True)
)
Total number of trainable parameters: 338


Run 1/3
Fold 1/3
Fold 1 Results:
   Accuracy: 0.6444
   Average Sensitivity (Recall): 0.6472
   Average Specificity: 0.6472
Fold 2/3
Fold 2 Results:
   Accuracy: 0.5111
   Average Sensitivity (Recall): 0.4700
   Average Specificity: 0.4700
Fold 3/3
Fold 3 Results:
   Accuracy: 0.5227
   Average Sensitivity (Recall): 0.5547
   Average Specificity: 0.5547
Run 1 Cross-Validation Results:
accuracy: Mean = 0.5594, Std = 0.0603
sensitivity: Mean = 0.5573, Std = 0.0724
specificity: Mean = 0.5573, Std = 0.0724

Run 2/3
Fold 1/3
Fold 1 Results:
   Accuracy: 0.5333
   Average Sensitivity (Recall): 0.5385
   Average Specificity: 0.5385
Fold 2/3
Fold 2 Results:
   Accuracy: 0.5778
   Average Sensitivity (Recall): 0.5750
   Average Speci

In [90]:
set_seed(42)
gcesn_emci = GCESN_1layer(emci_num_features, 2*emci_num_features, emci_num_classes, leaky_rate=0.8, num_iterations=5)
print(gcesn_emci)
print(f"Total number of trainable parameters: {gcesn_emci.count_parameters()}\n")

avg_metrics = multi_train_test_cv(gcesn_emci, emci_dataset,
                lr=0.001, num_epochs=500, step_size=500, gamma=0.5, 
                num_runs=3, binary_classification=False, 
                best_model_path='models/best_gcesn_1l_emci.pth')

GCESN_1layer(
  (fc1): Linear(in_features=16, out_features=16, bias=True)
  (bn): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc): Linear(in_features=16, out_features=2, bias=True)
)
Total number of trainable parameters: 338


Run 1/3
Fold 1/3
Fold 1 Results:
   Accuracy: 0.5556
   Average Sensitivity (Recall): 0.5573
   Average Specificity: 0.5573
Fold 2/3
Fold 2 Results:
   Accuracy: 0.5333
   Average Sensitivity (Recall): 0.4950
   Average Specificity: 0.4950
Fold 3/3
Fold 3 Results:
   Accuracy: 0.5455
   Average Sensitivity (Recall): 0.5747
   Average Specificity: 0.5747
Run 1 Cross-Validation Results:
accuracy: Mean = 0.5448, Std = 0.0091
sensitivity: Mean = 0.5423, Std = 0.0342
specificity: Mean = 0.5423, Std = 0.0342

Run 2/3
Fold 1/3
Fold 1 Results:
   Accuracy: 0.4889
   Average Sensitivity (Recall): 0.4911
   Average Specificity: 0.4911
Fold 2/3
Fold 2 Results:
   Accuracy: 0.5778
   Average Sensitivity (Recall): 0.5900
   Average Speci

In [91]:
set_seed(42)
gcesn_emci = GCESN_1layer(emci_num_features, 2*emci_num_features, emci_num_classes, leaky_rate=0.7, num_iterations=5)
print(gcesn_emci)
print(f"Total number of trainable parameters: {gcesn_emci.count_parameters()}\n")

avg_metrics = multi_train_test_cv(gcesn_emci, emci_dataset,
                lr=0.001, num_epochs=500, step_size=500, gamma=0.5, 
                num_runs=3, binary_classification=False, 
                best_model_path='models/best_gcesn_1l_emci.pth')

GCESN_1layer(
  (fc1): Linear(in_features=16, out_features=16, bias=True)
  (bn): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc): Linear(in_features=16, out_features=2, bias=True)
)
Total number of trainable parameters: 338


Run 1/3
Fold 1/3
Fold 1 Results:
   Accuracy: 0.5556
   Average Sensitivity (Recall): 0.5593
   Average Specificity: 0.5593
Fold 2/3
Fold 2 Results:
   Accuracy: 0.4667
   Average Sensitivity (Recall): 0.4900
   Average Specificity: 0.4900
Fold 3/3
Fold 3 Results:
   Accuracy: 0.5000
   Average Sensitivity (Recall): 0.5474
   Average Specificity: 0.5474
Run 1 Cross-Validation Results:
accuracy: Mean = 0.5074, Std = 0.0367
sensitivity: Mean = 0.5322, Std = 0.0302
specificity: Mean = 0.5322, Std = 0.0302

Run 2/3
Fold 1/3
Fold 1 Results:
   Accuracy: 0.4889
   Average Sensitivity (Recall): 0.4852
   Average Specificity: 0.4852
Fold 2/3
Fold 2 Results:
   Accuracy: 0.4444
   Average Sensitivity (Recall): 0.4750
   Average Speci

## Trainable GCESN

### Iteration 1

In [93]:
set_seed(42)
trainable_gcesn_emci = TrainableGCESN_1layer(emci_num_features, 2*emci_num_features, emci_num_classes, 
                        leaky_rate=1, num_iterations=1)
print(f"Total number of trainable parameters: {trainable_gcesn_emci.count_parameters()}\n")
                
avg_metrics = single_train_test_cv(trainable_gcesn_emci, emci_dataset,
                lr=0.005, num_epochs=500, step_size=500,
                save_path='models/trainable_gcesn_1l_emci.pth',
                binary_classification=False, is_esn=True)

Total number of trainable parameters: 722


Fold 1/3
Fold 1 Results:
   Accuracy: 0.4444
   Average Sensitivity (Recall): 0.4358
   Average Specificity: 0.4358

Fold 2/3
Fold 2 Results:
   Accuracy: 0.4667
   Average Sensitivity (Recall): 0.5100
   Average Specificity: 0.5100

Fold 3/3
Fold 3 Results:
   Accuracy: 0.4318
   Average Sensitivity (Recall): 0.4558
   Average Specificity: 0.4558

Cross-Validation Results:
accuracy: Mean = 0.4476 ± 0.01
sensitivity: Mean = 0.4672 ± 0.03
specificity: Mean = 0.4672 ± 0.03
epochs: Mean = 162.6667 ± 25.09
training_times: Mean = 1.7590 ± 0.26
testing_times: Mean = 0.0041 ± 0.00
avg_memory_usage: Mean = 0.1380 ± 0.00
avg_gpu_usage: Mean = 0.0000 ± 0.00
max_memory_usage: Mean = 0.1390 ± 0.00
max_gpu_usage: Mean = 0.0000 ± 0.00


In [94]:
set_seed(42)
trainable_gcesn_emci = TrainableGCESN_1layer(emci_num_features, 2*emci_num_features, emci_num_classes, 
                        leaky_rate=0.9, num_iterations=1)
print(f"Total number of trainable parameters: {trainable_gcesn_emci.count_parameters()}\n")
                
avg_metrics = single_train_test_cv(trainable_gcesn_emci, emci_dataset,
                lr=0.005, num_epochs=500, step_size=500,
                save_path='models/trainable_gcesn_1l_emci.pth',
                binary_classification=False, is_esn=True)

Total number of trainable parameters: 722


Fold 1/3
Fold 1 Results:
   Accuracy: 0.4222
   Average Sensitivity (Recall): 0.4140
   Average Specificity: 0.4140

Fold 2/3
Fold 2 Results:
   Accuracy: 0.6000
   Average Sensitivity (Recall): 0.5650
   Average Specificity: 0.5650

Fold 3/3
Fold 3 Results:
   Accuracy: 0.4318
   Average Sensitivity (Recall): 0.5000
   Average Specificity: 0.5000

Cross-Validation Results:
accuracy: Mean = 0.4847 ± 0.08
sensitivity: Mean = 0.4930 ± 0.06
specificity: Mean = 0.4930 ± 0.06
epochs: Mean = 131.3333 ± 8.96
training_times: Mean = 1.4846 ± 0.16
testing_times: Mean = 0.0040 ± 0.00
avg_memory_usage: Mean = 0.1440 ± 0.01
avg_gpu_usage: Mean = 0.0000 ± 0.00
max_memory_usage: Mean = 0.1485 ± 0.00
max_gpu_usage: Mean = 0.0000 ± 0.00


In [95]:
set_seed(42)
trainable_gcesn_emci = TrainableGCESN_1layer(emci_num_features, 2*emci_num_features, emci_num_classes, 
                        leaky_rate=0.8, num_iterations=1)
print(f"Total number of trainable parameters: {trainable_gcesn_emci.count_parameters()}\n")
                
avg_metrics = single_train_test_cv(trainable_gcesn_emci, emci_dataset,
                lr=0.005, num_epochs=500, step_size=500,
                save_path='models/trainable_gcesn_1l_emci.pth',
                binary_classification=False, is_esn=True)

Total number of trainable parameters: 722


Fold 1/3
Fold 1 Results:
   Accuracy: 0.5111
   Average Sensitivity (Recall): 0.5010
   Average Specificity: 0.5010

Fold 2/3
Fold 2 Results:
   Accuracy: 0.4222
   Average Sensitivity (Recall): 0.4750
   Average Specificity: 0.4750

Fold 3/3
Fold 3 Results:
   Accuracy: 0.4091
   Average Sensitivity (Recall): 0.4611
   Average Specificity: 0.4611

Cross-Validation Results:
accuracy: Mean = 0.4475 ± 0.05
sensitivity: Mean = 0.4790 ± 0.02
specificity: Mean = 0.4790 ± 0.02
epochs: Mean = 153.3333 ± 20.76
training_times: Mean = 1.7522 ± 0.26
testing_times: Mean = 0.0041 ± 0.00
avg_memory_usage: Mean = 0.1551 ± 0.00
avg_gpu_usage: Mean = 0.0000 ± 0.00
max_memory_usage: Mean = 0.1567 ± 0.00
max_gpu_usage: Mean = 0.0000 ± 0.00


In [96]:
set_seed(42)
trainable_gcesn_emci = TrainableGCESN_1layer(emci_num_features, 2*emci_num_features, emci_num_classes, 
                        leaky_rate=0.7, num_iterations=1)
print(f"Total number of trainable parameters: {trainable_gcesn_emci.count_parameters()}\n")
                
avg_metrics = single_train_test_cv(trainable_gcesn_emci, emci_dataset,
                lr=0.005, num_epochs=500, step_size=500,
                save_path='models/trainable_gcesn_1l_emci.pth',
                binary_classification=False, is_esn=True)

Total number of trainable parameters: 722


Fold 1/3
Fold 1 Results:
   Accuracy: 0.4889
   Average Sensitivity (Recall): 0.4822
   Average Specificity: 0.4822

Fold 2/3
Fold 2 Results:
   Accuracy: 0.4667
   Average Sensitivity (Recall): 0.5150
   Average Specificity: 0.5150

Fold 3/3
Fold 3 Results:
   Accuracy: 0.4318
   Average Sensitivity (Recall): 0.5000
   Average Specificity: 0.5000

Cross-Validation Results:
accuracy: Mean = 0.4625 ± 0.02
sensitivity: Mean = 0.4991 ± 0.01
specificity: Mean = 0.4991 ± 0.01
epochs: Mean = 170.6667 ± 26.95
training_times: Mean = 1.8903 ± 0.28
testing_times: Mean = 0.0041 ± 0.00
avg_memory_usage: Mean = 0.1632 ± 0.00
avg_gpu_usage: Mean = 0.0000 ± 0.00
max_memory_usage: Mean = 0.1645 ± 0.00
max_gpu_usage: Mean = 0.0000 ± 0.00


### Iteration 2

In [112]:
set_seed(42)
trainable_gcesn_emci = TrainableGCESN_1layer(emci_num_features, 2*emci_num_features, emci_num_classes, 
                        leaky_rate=1, num_iterations=2)
print(f"Total number of trainable parameters: {trainable_gcesn_emci.count_parameters()}\n")
                
avg_metrics = single_train_test_cv(trainable_gcesn_emci, emci_dataset,
                lr=0.005, num_epochs=500, step_size=500,
                save_path='models/trainable_gcesn_1l_emci.pth',
                binary_classification=False, is_esn=True)

Total number of trainable parameters: 722


Fold 1/3
Fold 1 Results:
   Accuracy: 0.4667
   Average Sensitivity (Recall): 0.4753
   Average Specificity: 0.4753

Fold 2/3
Fold 2 Results:
   Accuracy: 0.5556
   Average Sensitivity (Recall): 0.5000
   Average Specificity: 0.5000

Fold 3/3
Fold 3 Results:
   Accuracy: 0.4773
   Average Sensitivity (Recall): 0.5147
   Average Specificity: 0.5147

Cross-Validation Results:
accuracy: Mean = 0.4998 ± 0.04
sensitivity: Mean = 0.4967 ± 0.02
specificity: Mean = 0.4967 ± 0.02
epochs: Mean = 89.3333 ± 14.97
training_times: Mean = 1.1437 ± 0.20
testing_times: Mean = 0.0044 ± 0.00
avg_memory_usage: Mean = 0.1402 ± 0.00
avg_gpu_usage: Mean = 0.0000 ± 0.00
max_memory_usage: Mean = 0.1453 ± 0.00
max_gpu_usage: Mean = 0.0000 ± 0.00


In [111]:
set_seed(42)
trainable_gcesn_emci = TrainableGCESN_1layer(emci_num_features, 2*emci_num_features, emci_num_classes, 
                        leaky_rate=0.9, num_iterations=2)
print(f"Total number of trainable parameters: {trainable_gcesn_emci.count_parameters()}\n")
                
avg_metrics = single_train_test_cv(trainable_gcesn_emci, emci_dataset,
                lr=0.005, num_epochs=500, step_size=500,
                save_path='models/trainable_gcesn_1l_emci.pth',
                binary_classification=False, is_esn=True)

Total number of trainable parameters: 722


Fold 1/3
Fold 1 Results:
   Accuracy: 0.5111
   Average Sensitivity (Recall): 0.5079
   Average Specificity: 0.5079

Fold 2/3
Fold 2 Results:
   Accuracy: 0.5333
   Average Sensitivity (Recall): 0.5450
   Average Specificity: 0.5450

Fold 3/3
Fold 3 Results:
   Accuracy: 0.4318
   Average Sensitivity (Recall): 0.5000
   Average Specificity: 0.5000

Cross-Validation Results:
accuracy: Mean = 0.4921 ± 0.04
sensitivity: Mean = 0.5176 ± 0.02
specificity: Mean = 0.5176 ± 0.02
epochs: Mean = 214.6667 ± 201.89
training_times: Mean = 2.6813 ± 2.56
testing_times: Mean = 0.0043 ± 0.00
avg_memory_usage: Mean = 0.1421 ± 0.00
avg_gpu_usage: Mean = 0.0000 ± 0.00
max_memory_usage: Mean = 0.1463 ± 0.00
max_gpu_usage: Mean = 0.0000 ± 0.00


In [110]:
set_seed(42)
trainable_gcesn_emci = TrainableGCESN_1layer(emci_num_features, 2*emci_num_features, emci_num_classes, 
                        leaky_rate=0.8, num_iterations=2)
print(f"Total number of trainable parameters: {trainable_gcesn_emci.count_parameters()}\n")
                
avg_metrics = single_train_test_cv(trainable_gcesn_emci, emci_dataset,
                lr=0.005, num_epochs=500, step_size=500,
                save_path='models/trainable_gcesn_1l_emci.pth',
                binary_classification=False, is_esn=True)

Total number of trainable parameters: 722


Fold 1/3
Fold 1 Results:
   Accuracy: 0.4667
   Average Sensitivity (Recall): 0.4684
   Average Specificity: 0.4684

Fold 2/3
Fold 2 Results:
   Accuracy: 0.4667
   Average Sensitivity (Recall): 0.5200
   Average Specificity: 0.5200

Fold 3/3
Fold 3 Results:
   Accuracy: 0.4773
   Average Sensitivity (Recall): 0.4895
   Average Specificity: 0.4895

Cross-Validation Results:
accuracy: Mean = 0.4702 ± 0.00
sensitivity: Mean = 0.4926 ± 0.02
specificity: Mean = 0.4926 ± 0.02
epochs: Mean = 224.0000 ± 196.69
training_times: Mean = 2.8341 ± 2.53
testing_times: Mean = 0.0049 ± 0.00
avg_memory_usage: Mean = 0.1414 ± 0.00
avg_gpu_usage: Mean = 0.0000 ± 0.00
max_memory_usage: Mean = 0.1461 ± 0.01
max_gpu_usage: Mean = 0.0000 ± 0.00


In [109]:
set_seed(42)
trainable_gcesn_emci = TrainableGCESN_1layer(emci_num_features, 2*emci_num_features, emci_num_classes, 
                        leaky_rate=0.7, num_iterations=2)
print(f"Total number of trainable parameters: {trainable_gcesn_emci.count_parameters()}\n")
                
avg_metrics = single_train_test_cv(trainable_gcesn_emci, emci_dataset,
                lr=0.005, num_epochs=500, step_size=500,
                save_path='models/trainable_gcesn_1l_emci.pth',
                binary_classification=False, is_esn=True)

Total number of trainable parameters: 722


Fold 1/3
Fold 1 Results:
   Accuracy: 0.5111
   Average Sensitivity (Recall): 0.5148
   Average Specificity: 0.5148

Fold 2/3
Fold 2 Results:
   Accuracy: 0.5778
   Average Sensitivity (Recall): 0.5850
   Average Specificity: 0.5850

Fold 3/3
Fold 3 Results:
   Accuracy: 0.2955
   Average Sensitivity (Recall): 0.3295
   Average Specificity: 0.3295

Cross-Validation Results:
accuracy: Mean = 0.4614 ± 0.12
sensitivity: Mean = 0.4764 ± 0.11
specificity: Mean = 0.4764 ± 0.11
epochs: Mean = 214.0000 ± 202.24
training_times: Mean = 2.7390 ± 2.64
testing_times: Mean = 0.0046 ± 0.00
avg_memory_usage: Mean = 0.1499 ± 0.00
avg_gpu_usage: Mean = 0.0000 ± 0.00
max_memory_usage: Mean = 0.1532 ± 0.00
max_gpu_usage: Mean = 0.0000 ± 0.00


### Iteration 3

In [108]:
set_seed(42)
trainable_gcesn_emci = TrainableGCESN_1layer(emci_num_features, 2*emci_num_features, emci_num_classes, 
                        leaky_rate=1, num_iterations=3)
print(f"Total number of trainable parameters: {trainable_gcesn_emci.count_parameters()}\n")
                
avg_metrics = single_train_test_cv(trainable_gcesn_emci, emci_dataset,
                lr=0.005, num_epochs=500, step_size=500,
                save_path='models/trainable_gcesn_1l_emci.pth',
                binary_classification=False, is_esn=True)

Total number of trainable parameters: 722


Fold 1/3
Fold 1 Results:
   Accuracy: 0.4000
   Average Sensitivity (Recall): 0.4022
   Average Specificity: 0.4022

Fold 2/3
Fold 2 Results:
   Accuracy: 0.5111
   Average Sensitivity (Recall): 0.4600
   Average Specificity: 0.4600

Fold 3/3
Fold 3 Results:
   Accuracy: 0.4318
   Average Sensitivity (Recall): 0.5000
   Average Specificity: 0.5000

Cross-Validation Results:
accuracy: Mean = 0.4476 ± 0.05
sensitivity: Mean = 0.4541 ± 0.04
specificity: Mean = 0.4541 ± 0.04
epochs: Mean = 206.3333 ± 208.26
training_times: Mean = 2.8682 ± 2.94
testing_times: Mean = 0.0049 ± 0.00
avg_memory_usage: Mean = 0.1472 ± 0.01
avg_gpu_usage: Mean = 0.0000 ± 0.00
max_memory_usage: Mean = 0.1498 ± 0.01
max_gpu_usage: Mean = 0.0000 ± 0.00


In [107]:
set_seed(42)
trainable_gcesn_emci = TrainableGCESN_1layer(emci_num_features, 2*emci_num_features, emci_num_classes, 
                        leaky_rate=0.9, num_iterations=3)
print(f"Total number of trainable parameters: {trainable_gcesn_emci.count_parameters()}\n")
                
avg_metrics = single_train_test_cv(trainable_gcesn_emci, emci_dataset,
                lr=0.005, num_epochs=500, step_size=500,
                save_path='models/trainable_gcesn_1l_emci.pth',
                binary_classification=False, is_esn=True)

Total number of trainable parameters: 722


Fold 1/3
Fold 1 Results:
   Accuracy: 0.4667
   Average Sensitivity (Recall): 0.4743
   Average Specificity: 0.4743

Fold 2/3
Fold 2 Results:
   Accuracy: 0.4444
   Average Sensitivity (Recall): 0.5000
   Average Specificity: 0.5000

Fold 3/3
Fold 3 Results:
   Accuracy: 0.5455
   Average Sensitivity (Recall): 0.5684
   Average Specificity: 0.5684

Cross-Validation Results:
accuracy: Mean = 0.4855 ± 0.04
sensitivity: Mean = 0.5142 ± 0.04
specificity: Mean = 0.5142 ± 0.04
epochs: Mean = 231.3333 ± 190.48
training_times: Mean = 3.1099 ± 2.50
testing_times: Mean = 0.0047 ± 0.00
avg_memory_usage: Mean = 0.1457 ± 0.00
avg_gpu_usage: Mean = 0.0000 ± 0.00
max_memory_usage: Mean = 0.1496 ± 0.00
max_gpu_usage: Mean = 0.0000 ± 0.00


In [106]:
set_seed(42)
trainable_gcesn_emci = TrainableGCESN_1layer(emci_num_features, 2*emci_num_features, emci_num_classes, 
                        leaky_rate=0.8, num_iterations=3)
print(f"Total number of trainable parameters: {trainable_gcesn_emci.count_parameters()}\n")
                
avg_metrics = single_train_test_cv(trainable_gcesn_emci, emci_dataset,
                lr=0.005, num_epochs=500, step_size=500,
                save_path='models/trainable_gcesn_1l_emci.pth',
                binary_classification=False, is_esn=True)

Total number of trainable parameters: 722


Fold 1/3
Fold 1 Results:
   Accuracy: 0.4444
   Average Sensitivity (Recall): 0.4516
   Average Specificity: 0.4516

Fold 2/3
Fold 2 Results:
   Accuracy: 0.6000
   Average Sensitivity (Recall): 0.5950
   Average Specificity: 0.5950

Fold 3/3
Fold 3 Results:
   Accuracy: 0.4318
   Average Sensitivity (Recall): 0.5000
   Average Specificity: 0.5000

Cross-Validation Results:
accuracy: Mean = 0.4921 ± 0.08
sensitivity: Mean = 0.5155 ± 0.06
specificity: Mean = 0.5155 ± 0.06
epochs: Mean = 66.0000 ± 16.06
training_times: Mean = 0.8995 ± 0.21
testing_times: Mean = 0.0048 ± 0.00
avg_memory_usage: Mean = 0.1461 ± 0.00
avg_gpu_usage: Mean = 0.0000 ± 0.00
max_memory_usage: Mean = 0.1467 ± 0.00
max_gpu_usage: Mean = 0.0000 ± 0.00


In [105]:
set_seed(42)
trainable_gcesn_emci = TrainableGCESN_1layer(emci_num_features, 2*emci_num_features, emci_num_classes, 
                        leaky_rate=0.7, num_iterations=3)
print(f"Total number of trainable parameters: {trainable_gcesn_emci.count_parameters()}\n")
                
avg_metrics = single_train_test_cv(trainable_gcesn_emci, emci_dataset,
                lr=0.005, num_epochs=500, step_size=500,
                save_path='models/trainable_gcesn_1l_emci.pth',
                binary_classification=False, is_esn=True)

Total number of trainable parameters: 722


Fold 1/3
Fold 1 Results:
   Accuracy: 0.5333
   Average Sensitivity (Recall): 0.5356
   Average Specificity: 0.5356

Fold 2/3
Fold 2 Results:
   Accuracy: 0.5778
   Average Sensitivity (Recall): 0.5450
   Average Specificity: 0.5450

Fold 3/3
Fold 3 Results:
   Accuracy: 0.5909
   Average Sensitivity (Recall): 0.5389
   Average Specificity: 0.5389

Cross-Validation Results:
accuracy: Mean = 0.5673 ± 0.02
sensitivity: Mean = 0.5398 ± 0.00
specificity: Mean = 0.5398 ± 0.00
epochs: Mean = 117.6667 ± 36.94
training_times: Mean = 1.6519 ± 0.56
testing_times: Mean = 0.0046 ± 0.00
avg_memory_usage: Mean = 0.1442 ± 0.00
avg_gpu_usage: Mean = 0.0000 ± 0.00
max_memory_usage: Mean = 0.1464 ± 0.00
max_gpu_usage: Mean = 0.0000 ± 0.00


### Iteration 4

In [104]:
set_seed(42)
trainable_gcesn_emci = TrainableGCESN_1layer(emci_num_features, 2*emci_num_features, emci_num_classes, 
                        leaky_rate=1, num_iterations=4)
print(f"Total number of trainable parameters: {trainable_gcesn_emci.count_parameters()}\n")
                
avg_metrics = single_train_test_cv(trainable_gcesn_emci, emci_dataset,
                lr=0.005, num_epochs=500, step_size=500,
                save_path='models/trainable_gcesn_1l_emci.pth',
                binary_classification=False, is_esn=True)

Total number of trainable parameters: 722


Fold 1/3
Fold 1 Results:
   Accuracy: 0.4000
   Average Sensitivity (Recall): 0.3982
   Average Specificity: 0.3982

Fold 2/3
Fold 2 Results:
   Accuracy: 0.4667
   Average Sensitivity (Recall): 0.5000
   Average Specificity: 0.5000

Fold 3/3
Fold 3 Results:
   Accuracy: 0.3864
   Average Sensitivity (Recall): 0.4032
   Average Specificity: 0.4032

Cross-Validation Results:
accuracy: Mean = 0.4177 ± 0.04
sensitivity: Mean = 0.4338 ± 0.05
specificity: Mean = 0.4338 ± 0.05
epochs: Mean = 65.0000 ± 8.04
training_times: Mean = 0.9668 ± 0.14
testing_times: Mean = 0.0051 ± 0.00
avg_memory_usage: Mean = 0.1460 ± 0.00
avg_gpu_usage: Mean = 0.0000 ± 0.00
max_memory_usage: Mean = 0.1475 ± 0.00
max_gpu_usage: Mean = 0.0000 ± 0.00


In [103]:
set_seed(42)
trainable_gcesn_emci = TrainableGCESN_1layer(emci_num_features, 2*emci_num_features, emci_num_classes, 
                        leaky_rate=0.9, num_iterations=4)
print(f"Total number of trainable parameters: {trainable_gcesn_emci.count_parameters()}\n")
                
avg_metrics = single_train_test_cv(trainable_gcesn_emci, emci_dataset,
                lr=0.005, num_epochs=500, step_size=500,
                save_path='models/trainable_gcesn_1l_emci.pth',
                binary_classification=False, is_esn=True)

Total number of trainable parameters: 722


Fold 1/3
Fold 1 Results:
   Accuracy: 0.4444
   Average Sensitivity (Recall): 0.4506
   Average Specificity: 0.4506

Fold 2/3
Fold 2 Results:
   Accuracy: 0.5556
   Average Sensitivity (Recall): 0.5000
   Average Specificity: 0.5000

Fold 3/3
Fold 3 Results:
   Accuracy: 0.3636
   Average Sensitivity (Recall): 0.3832
   Average Specificity: 0.3832

Cross-Validation Results:
accuracy: Mean = 0.4545 ± 0.08
sensitivity: Mean = 0.4446 ± 0.05
specificity: Mean = 0.4446 ± 0.05
epochs: Mean = 217.0000 ± 200.37
training_times: Mean = 3.2485 ± 3.07
testing_times: Mean = 0.0047 ± 0.00
avg_memory_usage: Mean = 0.1557 ± 0.00
avg_gpu_usage: Mean = 0.0000 ± 0.00
max_memory_usage: Mean = 0.1567 ± 0.00
max_gpu_usage: Mean = 0.0000 ± 0.00


In [102]:
set_seed(42)
trainable_gcesn_emci = TrainableGCESN_1layer(emci_num_features, 2*emci_num_features, emci_num_classes, 
                        leaky_rate=0.8, num_iterations=4)
print(f"Total number of trainable parameters: {trainable_gcesn_emci.count_parameters()}\n")
                
avg_metrics = single_train_test_cv(trainable_gcesn_emci, emci_dataset,
                lr=0.005, num_epochs=500, step_size=500,
                save_path='models/trainable_gcesn_1l_emci.pth',
                binary_classification=False, is_esn=True)

Total number of trainable parameters: 722


Fold 1/3
Fold 1 Results:
   Accuracy: 0.4667
   Average Sensitivity (Recall): 0.4773
   Average Specificity: 0.4773

Fold 2/3
Fold 2 Results:
   Accuracy: 0.4444
   Average Sensitivity (Recall): 0.5000
   Average Specificity: 0.5000

Fold 3/3
Fold 3 Results:
   Accuracy: 0.4091
   Average Sensitivity (Recall): 0.4737
   Average Specificity: 0.4737

Cross-Validation Results:
accuracy: Mean = 0.4401 ± 0.02
sensitivity: Mean = 0.4837 ± 0.01
specificity: Mean = 0.4837 ± 0.01
epochs: Mean = 63.3333 ± 19.60
training_times: Mean = 0.9192 ± 0.26
testing_times: Mean = 0.0051 ± 0.00
avg_memory_usage: Mean = 0.1579 ± 0.00
avg_gpu_usage: Mean = 0.0000 ± 0.00
max_memory_usage: Mean = 0.1582 ± 0.00
max_gpu_usage: Mean = 0.0000 ± 0.00


In [101]:
set_seed(42)
trainable_gcesn_emci = TrainableGCESN_1layer(emci_num_features, 2*emci_num_features, emci_num_classes, 
                        leaky_rate=0.7, num_iterations=4)
print(f"Total number of trainable parameters: {trainable_gcesn_emci.count_parameters()}\n")
                
avg_metrics = single_train_test_cv(trainable_gcesn_emci, emci_dataset,
                lr=0.005, num_epochs=500, step_size=500,
                save_path='models/trainable_gcesn_1l_emci.pth',
                binary_classification=False, is_esn=True)

Total number of trainable parameters: 722


Fold 1/3
Fold 1 Results:
   Accuracy: 0.4667
   Average Sensitivity (Recall): 0.4753
   Average Specificity: 0.4753

Fold 2/3
Fold 2 Results:
   Accuracy: 0.5111
   Average Sensitivity (Recall): 0.5350
   Average Specificity: 0.5350

Fold 3/3
Fold 3 Results:
   Accuracy: 0.4318
   Average Sensitivity (Recall): 0.5000
   Average Specificity: 0.5000

Cross-Validation Results:
accuracy: Mean = 0.4699 ± 0.03
sensitivity: Mean = 0.5034 ± 0.02
specificity: Mean = 0.5034 ± 0.02
epochs: Mean = 75.0000 ± 19.65
training_times: Mean = 1.1112 ± 0.28
testing_times: Mean = 0.0051 ± 0.00
avg_memory_usage: Mean = 0.1559 ± 0.00
avg_gpu_usage: Mean = 0.0000 ± 0.00
max_memory_usage: Mean = 0.1571 ± 0.00
max_gpu_usage: Mean = 0.0000 ± 0.00


### Iteration 5

In [100]:
set_seed(42)
trainable_gcesn_emci = TrainableGCESN_1layer(emci_num_features, 2*emci_num_features, emci_num_classes, 
                        leaky_rate=1, num_iterations=5)
print(f"Total number of trainable parameters: {trainable_gcesn_emci.count_parameters()}\n")
                
avg_metrics = single_train_test_cv(trainable_gcesn_emci, emci_dataset,
                lr=0.005, num_epochs=500, step_size=500,
                save_path='models/trainable_gcesn_1l_emci.pth',
                binary_classification=False, is_esn=True)

Total number of trainable parameters: 722


Fold 1/3
Fold 1 Results:
   Accuracy: 0.4889
   Average Sensitivity (Recall): 0.5000
   Average Specificity: 0.5000

Fold 2/3
Fold 2 Results:
   Accuracy: 0.5333
   Average Sensitivity (Recall): 0.5300
   Average Specificity: 0.5300

Fold 3/3
Fold 3 Results:
   Accuracy: 0.4091
   Average Sensitivity (Recall): 0.4737
   Average Specificity: 0.4737

Cross-Validation Results:
accuracy: Mean = 0.4771 ± 0.05
sensitivity: Mean = 0.5012 ± 0.02
specificity: Mean = 0.5012 ± 0.02
epochs: Mean = 60.0000 ± 4.32
training_times: Mean = 0.9837 ± 0.06
testing_times: Mean = 0.0055 ± 0.00
avg_memory_usage: Mean = 0.1647 ± 0.00
avg_gpu_usage: Mean = 0.0000 ± 0.00
max_memory_usage: Mean = 0.1681 ± 0.00
max_gpu_usage: Mean = 0.0000 ± 0.00


In [99]:
set_seed(42)
trainable_gcesn_emci = TrainableGCESN_1layer(emci_num_features, 2*emci_num_features, emci_num_classes, 
                        leaky_rate=0.9, num_iterations=5)
print(f"Total number of trainable parameters: {trainable_gcesn_emci.count_parameters()}\n")
                
avg_metrics = single_train_test_cv(trainable_gcesn_emci, emci_dataset,
                lr=0.005, num_epochs=500, step_size=500,
                save_path='models/trainable_gcesn_1l_emci.pth',
                binary_classification=False, is_esn=True)

Total number of trainable parameters: 722


Fold 1/3
Fold 1 Results:
   Accuracy: 0.4444
   Average Sensitivity (Recall): 0.4516
   Average Specificity: 0.4516

Fold 2/3
Fold 2 Results:
   Accuracy: 0.5556
   Average Sensitivity (Recall): 0.5000
   Average Specificity: 0.5000

Fold 3/3
Fold 3 Results:
   Accuracy: 0.4545
   Average Sensitivity (Recall): 0.5200
   Average Specificity: 0.5200

Cross-Validation Results:
accuracy: Mean = 0.4848 ± 0.05
sensitivity: Mean = 0.4905 ± 0.03
specificity: Mean = 0.4905 ± 0.03
epochs: Mean = 60.0000 ± 17.28
training_times: Mean = 0.9464 ± 0.28
testing_times: Mean = 0.0055 ± 0.00
avg_memory_usage: Mean = 0.1693 ± 0.00
avg_gpu_usage: Mean = 0.0000 ± 0.00
max_memory_usage: Mean = 0.1693 ± 0.00
max_gpu_usage: Mean = 0.0000 ± 0.00


In [98]:
set_seed(42)
trainable_gcesn_emci = TrainableGCESN_1layer(emci_num_features, 2*emci_num_features, emci_num_classes, 
                        leaky_rate=0.8, num_iterations=5)
print(f"Total number of trainable parameters: {trainable_gcesn_emci.count_parameters()}\n")
                
avg_metrics = single_train_test_cv(trainable_gcesn_emci, emci_dataset,
                lr=0.005, num_epochs=500, step_size=500,
                save_path='models/trainable_gcesn_1l_emci.pth',
                binary_classification=False, is_esn=True)

Total number of trainable parameters: 722


Fold 1/3
Fold 1 Results:
   Accuracy: 0.5111
   Average Sensitivity (Recall): 0.5000
   Average Specificity: 0.5000

Fold 2/3
Fold 2 Results:
   Accuracy: 0.4444
   Average Sensitivity (Recall): 0.5000
   Average Specificity: 0.5000

Fold 3/3
Fold 3 Results:
   Accuracy: 0.4545
   Average Sensitivity (Recall): 0.5074
   Average Specificity: 0.5074

Cross-Validation Results:
accuracy: Mean = 0.4700 ± 0.03
sensitivity: Mean = 0.5025 ± 0.00
specificity: Mean = 0.5025 ± 0.00
epochs: Mean = 68.6667 ± 8.73
training_times: Mean = 1.0382 ± 0.15
testing_times: Mean = 0.0051 ± 0.00
avg_memory_usage: Mean = 0.1689 ± 0.00
avg_gpu_usage: Mean = 0.0000 ± 0.00
max_memory_usage: Mean = 0.1689 ± 0.00
max_gpu_usage: Mean = 0.0000 ± 0.00


In [97]:
set_seed(42)
trainable_gcesn_emci_2 = TrainableGCESN_2layer(emci_num_features, 2*emci_num_features, emci_num_classes,
                        leaky_rate=0.7, num_iterations=5)
print(f"Total number of trainable parameters: {trainable_gcesn_emci_2.count_parameters()}\n")
                
avg_metrics = single_train_test_cv(trainable_gcesn_emci, emci_dataset,
                lr=0.005, num_epochs=500, step_size=500,
                save_path='models/trainable_gcesn_1l_emci.pth',
                binary_classification=False, is_esn=True)

Total number of trainable parameters: 2546


Fold 1/3
Fold 1 Results:
   Accuracy: 0.4889
   Average Sensitivity (Recall): 0.4822
   Average Specificity: 0.4822

Fold 2/3
Fold 2 Results:
   Accuracy: 0.4667
   Average Sensitivity (Recall): 0.5150
   Average Specificity: 0.5150

Fold 3/3
Fold 3 Results:
   Accuracy: 0.4318
   Average Sensitivity (Recall): 0.5000
   Average Specificity: 0.5000

Cross-Validation Results:
accuracy: Mean = 0.4625 ± 0.02
sensitivity: Mean = 0.4991 ± 0.01
specificity: Mean = 0.4991 ± 0.01
epochs: Mean = 170.6667 ± 26.95
training_times: Mean = 1.9455 ± 0.30
testing_times: Mean = 0.0039 ± 0.00
avg_memory_usage: Mean = 0.1686 ± 0.00
avg_gpu_usage: Mean = 0.0000 ± 0.00
max_memory_usage: Mean = 0.1686 ± 0.00
max_gpu_usage: Mean = 0.0000 ± 0.00
